<a href="https://colab.research.google.com/github/Untick/Marketplaces/blob/Kosmylev-Vladislav-branch/3_svod_DataSet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Подготовка

## Загрузка исходников

####**Загрузка библиотек:**

https://drive.google.com/file/d/1-_LKbDsiq0dDbks7_JnBJVV7v2iWjSkd/view?usp=sharing

https://drive.google.com/file/d/1CzIZl5hxOpNsEiWkcJgjXRsAVJz4RbsX/view?usp=sharing  target_tab.zip

In [ ]:
import numpy as np                        # Импорт библиотеки numpy
import pandas as pd                       # для анализа табличных массивов данных
from pandas.core.groupby import NamedAgg
import gdown, os, re, random                  # Стандартные библиотеки

In [ ]:
!gdown --id '1-_LKbDsiq0dDbks7_JnBJVV7v2iWjSkd'
#!gdown --id '1CzIZl5hxOpNsEiWkcJgjXRsAVJz4RbsX'
!rm -r sample_data
#!rm base_for_analiz.zip
!unzip -o /content/base_for_analiz.zip
# !unzip -o /content/target_tab.zip
# !rm target_tab.zip
!rm base_for_analiz.zip

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1-_LKbDsiq0dDbks7_JnBJVV7v2iWjSkd
To: /content/base_for_analiz.zip
100% 88.5M/88.5M [00:01<00:00, 83.5MB/s]
Archive:  /content/base_for_analiz.zip
   creating: base_for_analiz/
   creating: base_for_analiz/MPSTATS/
   creating: base_for_analiz/MPSTATS/Категория/
   creating: base_for_analiz/MPSTATS/Категория/MPSTATS Категория по дням/
  inflating: base_for_analiz/MPSTATS/Категория/MPSTATS Категория по дням/OZON - Автотовары_Автозапчасти_Автосвет - По дням - 01.03.2023-31.05.2023. (06.06.2023).csv  
  inflating: base_for_analiz/MPSTATS/Категория/MPSTATS Категория по дням/OZON - Автотовары_Автозапчасти_Автосвет - По дням - 01.06.2022-31.08.2022. (06.06.2023).csv  
  inflating: base_for_analiz/MPSTATS/Категория/MPSTATS Катего

In [ ]:
def travel_dir(folder_path, subdir=True):
    file_dict = {}
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            file_name = os.path.splitext(file)[0]
            file_dict[file_name] = file_path
        if not subdir:
          return file_dict
    return file_dict


In [ ]:
import zipfile
def travel_dir_zip(folder_path):
    # Создаем словарь, в котором будем хранить файлы
    files_dict = {}
    # Обходим папку с архивами
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        # Проверяем, является ли файл zip-архивом
        if zipfile.is_zipfile(file_path):
            # Разархивируем файл
            with zipfile.ZipFile(file_path, 'r') as zip_ref:
                zip_ref.extractall(folder_path)
            # Удаляем файл zip
            os.remove(file_path)
            # Обходим папку с разархивированными файлами
        for extracted_file_name in os.listdir(folder_path):
            extracted_file_path = os.path.join(folder_path, extracted_file_name)
            # Добавляем файл в словарь
            files_dict[extracted_file_name] = extracted_file_path
    return files_dict


In [ ]:
import re

def clean_column_names(df):
    cleaned_columns = []

    for column in df.columns:
        cleaned_column = re.sub(r"[^\w\s%]", "", column)  # Оставляем только буквы, цифры и пробелы
        cleaned_column = cleaned_column.replace(" ", "_")  # Заменяем пробелы на "_"
        if cleaned_column[-1] == '_':
            cleaned_column = cleaned_column[:-1]
        cleaned_columns.append(cleaned_column)
    return cleaned_columns
    # df.columns = cleaned_columns
    # return df

def normal_col(df):
  cols = clean_column_names(df)
  print(cols)
  df.set_axis(cols, axis=1, inplace=True)

In [ ]:
def float2int(df):
  col_float = list(df.select_dtypes(include=['float64']).columns)
  df.fillna(0, inplace=True)
  for col in col_float:
    df[col]=df[col].round(0).astype('int64')

In [ ]:
# Нормализуем заголовки столбцов
remove_list = ["(при наличии)",                 # список кусков текста для удаления из названий столбцов
               "(кроме Твери)",
               "руб.",
               "(перечеркнутая цена)",
               ",",
               "%",
               "с учетом скидки",
               "на рыночную цену"]
def clean_header(header):

    cleaned_header = header
    cleaned_header = cleaned_header.replace('-', "_")
    cleaned_header = cleaned_header.replace('" ', '"')
    cleaned_header = cleaned_header.replace(' "', '"')

    for item in remove_list:
        cleaned_header = cleaned_header.replace(item, "")

    cleaned_header = cleaned_header.replace('  ', '')
    cleaned_header = cleaned_header.replace('_"', '"')
    cleaned_header = cleaned_header.strip()  # удаление ведущих и концевых пробелов
    cleaned_header = cleaned_header.replace(" ", "_")  # замена оставшихся пробелов символом "_"
    cleaned_header = cleaned_header.replace('_"', '"')

    list_heads = cleaned_header.replace(';', ',')
    list_heads = list_heads.replace('"', '')
    list_heads = list_heads.replace('_"', '"')
    # print(header, " --->  ", list_heads)

    return cleaned_header

def clean_csv(file_path):
    with open(file_path, 'r', newline='') as file:
        lines = file.readlines()
        cleaned_headers = clean_header(lines[0]);  # применяем функцию clean_header к каждому заголовку
        lines[0] = cleaned_headers + '\n'
        print(cleaned_headers)

    with open(file_path, 'w', newline='') as file:
        file.writelines(lines)

    return cleaned_headers


#### Формируем ДатаФреймы

##### Пути по сегментам

In [ ]:
fdir = '/content/base_for_analiz/Ozon/Товары по категориям'
cat_paths = travel_dir(fdir)

fdir = '/content/base_for_analiz/Ozon/Performance/Трафареты'
traf_paths = travel_dir(fdir)

fdir = '/content/base_for_analiz/Ozon/Performance/ПВП'
pvp_paths = travel_dir(fdir)

fdir = '/content/base_for_analiz/Ozon'
ozon_paths = travel_dir(fdir, False)

fdir = '/content/base_for_analiz/MPSTATS/Категория/MPSTATS Категория по дням'
mpcat_day_paths = travel_dir(fdir)

fdir = '/content/base_for_analiz/MPSTATS/Категория/MPSTATS по категории (по всем продавцам)'
mpcat_sel_paths = travel_dir_zip(fdir)

fdir = '/content/base_for_analiz/MPSTATS/Магазин/MPSTATS 01062022-31052023 Выручка по дням'
mpmag_gros_paths = travel_dir(fdir)

fdir = '/content/base_for_analiz/MPSTATS/Магазин/MPSTATS 01062022-31052023 Товары'
mpmag_tov_paths = travel_dir(fdir)


### Загрузка исходников в ДатаФреймы

In [ ]:
all_df = {}

#### Ozon -> DF

загружаем индивидуально из-за особенностей данных

###### База

In [ ]:
os.getcwd()
os.chdir('/content/base_for_analiz/Ozon/')

In [ ]:
os.getcwd()

'/content/base_for_analiz/Ozon'

In [ ]:
columns = ['Артикул','Ozon_Product_ID','FBO_OZON_SKU_ID','FBS_OZON_SKU_ID','Контент_рейтинг','Текущая_цена']
xls = 'База.csv'
list_heads = clean_csv(xls)
df = pd.read_csv(xls, sep=';', encoding='utf-8', usecols=columns, decimal=',') #


﻿"Артикул";"Ozon_Product_ID";"FBO_OZON_SKU_ID";"FBS_OZON_SKU_ID";"Barcode";"Наименование_товара";"Контент_рейтинг";"Бренд";"Статус_товара";"Видимость_FBO";"Причины_скрытия_FBO";"Видимость_FBS";"Причины_скрытия_FBS";"Дата_создания";"Коммерческая_категория";"Объем_товара_л";"Объемный_вес_кг";"Доступно_на_складе_Ozon_шт";"Вывезти_и_нанести_КИЗшт";"Зарезервировано_шт";"Доступно_на_моих_складах_шт";"Зарезервировано_на_моих_складах_шт";"Текущая_цена";"Цена_до_скидки";"Цена_Premium";"Рыночная_цена";"Актуальная_ссылка";"Размер_НДС"


In [ ]:
df = df[df['Контент_рейтинг'] != "'Не рассчитываем контент-рейтинг для уценки FBS"].copy()

In [ ]:
df['Контент_рейтинг'] = df['Контент_рейтинг'].replace('Рассчитываем',"0.0", regex=True)
df['Контент_рейтинг'] = df['Контент_рейтинг'].replace("'","", regex=True)
df['Контент_рейтинг'] = df['Контент_рейтинг'].astype(float)
df['Артикул'] = df['Артикул'].replace("'","", regex=True).astype(str)
df['Ozon_Product_ID'] = df['Ozon_Product_ID'].astype(str)
df['FBO_OZON_SKU_ID'] = df['FBO_OZON_SKU_ID'].astype(str)
df['FBS_OZON_SKU_ID'] = df['FBS_OZON_SKU_ID'].astype(str)
df.rename(columns={"Ozon_Product_ID": "Ozon_ID",
                   "FBO_OZON_SKU_ID":"FBO_ID",
                   "FBS_OZON_SKU_ID":"FBS_ID"}, inplace=True)
float2int(df)

In [ ]:
df.dtypes

Артикул            object
Ozon_ID            object
FBO_ID             object
FBS_ID             object
Контент_рейтинг     int64
Текущая_цена        int64
dtype: object

In [ ]:
df.head(3)

,Артикул,Ozon_ID,FBO_ID,FBS_ID,Контент_рейтинг,Текущая_цена
0,30273,105606938,298131117,298131122,15,1200
1,30383,105742204,298264198,298264234,38,1990
2,31284,112525698,303785665,303785701,30,7900


In [ ]:
all_df['База'] = df

###### Графики

In [ ]:
xls = '/content/base_for_analiz/Ozon/Аналитика Графики 01.06.2022-31.05.2023.xlsx'
graf = pd.read_excel(xls)

In [ ]:
df = graf.copy()

In [ ]:
normal_col(df)


['Категория_3го_уровня', 'Товары', 'Модель', 'Ozon_ID', 'Артикул', 'День', 'Уникальные_посетители_всего', 'Уникальные_посетители_с_просмотром_карточки_товара', 'Показы_всего', 'Показы_на_карточке_товара', 'Показы_в_поиске_и_каталоге', 'Позиция_в_поиске_и_каталоге', 'В_корзину_всего', 'Заказано_товаров', 'Отменено_товаров', 'Доставлено_за_выбранный_период', 'Возвращено_товаров', 'Заказано_на_сумму', 'Отправления_с_премиумподпиской_принятые_в_обработку']


<ipython-input-5-06ccb71f2ffc>:19: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(cols, axis=1, inplace=True)


In [ ]:
def get_index(df):
  df['s_index'] = df['Ozon_ID'].astype(str) + df['День'].astype(str)

In [ ]:
get_index(df)

In [ ]:
df['Ozon_ID'] = df['Ozon_ID'].astype(str)
# graf['s_index'] = (graf['Ozon_ID']+graf['День']).replace('-','', regex=True).astype(str)
# Сборный индекс - для удобной группировки
#graf['s_index'] = ''.join(f'{graf['Ozon_ID']}_{graf['День']}').replace('-','', regex=True).astype(str)

df['День'] = pd.to_datetime(df['День'])
df['Позиция_в_поиске_и_каталоге'] = df['Позиция_в_поиске_и_каталоге'].replace(',','.', regex=True)
df['Позиция_в_поиске_и_каталоге'] = df['Позиция_в_поиске_и_каталоге'].astype(float)


In [ ]:
float2int(df)

In [ ]:
df.columns

Index(['Категория_3го_уровня', 'Товары', 'Модель', 'Ozon_ID', 'Артикул',
       'День', 'Уникальные_посетители_всего',
       'Уникальные_посетители_с_просмотром_карточки_товара', 'Показы_всего',
       'Показы_на_карточке_товара', 'Показы_в_поиске_и_каталоге',
       'Позиция_в_поиске_и_каталоге', 'В_корзину_всего', 'Заказано_товаров',
       'Отменено_товаров', 'Доставлено_за_выбранный_период',
       'Возвращено_товаров', 'Заказано_на_сумму',
       'Отправления_с_премиумподпиской_принятые_в_обработку', 's_index'],
      dtype='object')

In [ ]:
df.drop(['Товары', 'Модель',
       'Показы_на_карточке_товара', 'Показы_в_поиске_и_каталоге',
       'Отправления_с_премиумподпиской_принятые_в_обработку'], axis=1, inplace=True)

In [ ]:
df.dtypes

Категория_3го_уровня                                          object
Ozon_ID                                                       object
Артикул                                                       object
День                                                  datetime64[ns]
Уникальные_посетители_всего                                    int64
Уникальные_посетители_с_просмотром_карточки_товара             int64
Показы_всего                                                   int64
Позиция_в_поиске_и_каталоге                                    int64
В_корзину_всего                                                int64
Заказано_товаров                                               int64
Отменено_товаров                                               int64
Доставлено_за_выбранный_период                                 int64
Возвращено_товаров                                             int64
Заказано_на_сумму                                              int64
s_index                           

In [ ]:
all_df['Графики'] = df


In [ ]:
ddf = all_df['Графики']

In [ ]:
ddf.shape

(254748, 15)

In [ ]:
len(ddf['s_index'].unique())

254052

In [ ]:
ddf.shape[0] - len(ddf['s_index'].unique())

696

In [ ]:
ddf.columns

Index(['Категория_3го_уровня', 'Ozon_ID', 'Артикул', 'День',
       'Уникальные_посетители_всего',
       'Уникальные_посетители_с_просмотром_карточки_товара', 'Показы_всего',
       'Позиция_в_поиске_и_каталоге', 'В_корзину_всего', 'Заказано_товаров',
       'Отменено_товаров', 'Доставлено_за_выбранный_период',
       'Возвращено_товаров', 'Заказано_на_сумму', 's_index'],
      dtype='object')

Вывод: В базе есть не уникальные сочетания Ozon_ID и День

In [ ]:
#ddf.groupby(['День','Ozon_ID']).agg({'Уникальные_посетители_всего':'sum',
t = ddf.groupby(['s_index','Категория_3го_уровня', 'Ozon_ID', 'Артикул', 'День']).agg({'Уникальные_посетители_всего':'sum',
                                     'Уникальные_посетители_с_просмотром_карточки_товара':'sum',
                                     'Показы_всего':'sum',
                                     'Позиция_в_поиске_и_каталоге':'min',
                                     'В_корзину_всего':'sum',
                                     'Заказано_товаров':'sum',
                                     'Отменено_товаров':'sum',
                                     'Доставлено_за_выбранный_период':'sum',
                                     'Возвращено_товаров':'sum',
                                     'Заказано_на_сумму':'sum'}).reset_index()

In [ ]:
t.columns

Index(['s_index', 'Категория_3го_уровня', 'Ozon_ID', 'Артикул', 'День',
       'Уникальные_посетители_всего',
       'Уникальные_посетители_с_просмотром_карточки_товара', 'Показы_всего',
       'Позиция_в_поиске_и_каталоге', 'В_корзину_всего', 'Заказано_товаров',
       'Отменено_товаров', 'Доставлено_за_выбранный_период',
       'Возвращено_товаров', 'Заказано_на_сумму'],
      dtype='object')

In [ ]:
re_col = {'Показы_всего': 'Показы',
          'Уникальные_посетители_всего': 'Посетители',
          'Уникальные_посетители_с_просмотром_карточки_товара':'Посетители_на_карточке_товара',
          'В_корзину_всего': 'В_корзину',
          'Заказано_товаров':'Заказано',
          'Заказано_на_сумму':'Оборот_в_месяц_целевой',
          'Доставлено_за_выбранный_период':'Доставлено',
          'Возвращено_товаров':'Возвращено',
          'Отменено_товаров':'Отменено'
          }
t.rename(columns=re_col, inplace=True)

In [ ]:
t

,s_index,Категория_3го_уровня,Ozon_ID,Артикул,День,Посетители,Посетители_на_карточке_товара,Показы,Позиция_в_поиске_и_каталоге,В_корзину,Заказано,Отменено,Доставлено,Возвращено,Оборот_в_месяц_целевой
0,2609022712022-06-02,Лампа автомобильная,260902271,25541,2022-06-02,1,0,1,0,0,0,0,0,0,0
1,2609022712022-06-03,Лампа автомобильная,260902271,25541,2022-06-03,1,0,1,0,0,0,0,0,0,0
2,2609022712022-06-07,Лампа автомобильная,260902271,25541,2022-06-07,1,1,1,0,0,0,0,0,0,0
3,2609022712022-06-08,Лампа автомобильная,260902271,25541,2022-06-08,0,0,0,0,0,0,0,0,0,0
4,2609022712022-06-12,Лампа автомобильная,260902271,25541,2022-06-12,1,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254064,9660743052023-05-28,Лампа автомобильная,966074305,34599,2023-05-28,19,0,19,132,0,0,0,0,0,0
254065,9660743052023-05-29,Лампа автомобильная,966074305,34599,2023-05-29,26,1,28,155,0,0,0,0,0,0
254066,9660743052023-05-30,Лампа автомобильная,966074305,34599,2023-05-30,36,3,43,59,0,0,0,0,0,0
254067,9660743052023-05-31,Лампа автомобильная,966074305,34599,2023-05-31,21,3,30,144,1,0,0,0,0,0


In [ ]:
all_df['Графики'] = t

###### Акции  

In [ ]:
xls = '/content/base_for_analiz/Ozon/Аналитика Акции 01.06.2022-31.05.2023.xlsx'
act = pd.read_excel(xls)   #, encoding='cp1251' если выдаёт ошибку

In [ ]:
df = act.copy()
df.columns

Index(['Категория 3-го уровня', 'Товары', 'Модель', 'Ozon ID', 'Артикул',
       'Акции', 'День', 'Уникальные посетители, всего',
       'Уникальные посетители с просмотром карточки товара', 'Показы, всего',
       'Показы на карточке товара', 'Показы в поиске и каталоге',
       'Позиция в поиске и каталоге', 'В корзину, всего', 'Заказано товаров',
       'Доставлено (за выбранный период)', 'Заказано на сумму',
       'Заказано товаров по акции', 'Заказано на сумму по акции',
       'Отправления с премиум-подпиской, принятые в обработку'],
      dtype='object')

In [ ]:
col = normal_col(df)
df.columns

['Категория_3го_уровня', 'Товары', 'Модель', 'Ozon_ID', 'Артикул', 'Акции', 'День', 'Уникальные_посетители_всего', 'Уникальные_посетители_с_просмотром_карточки_товара', 'Показы_всего', 'Показы_на_карточке_товара', 'Показы_в_поиске_и_каталоге', 'Позиция_в_поиске_и_каталоге', 'В_корзину_всего', 'Заказано_товаров', 'Доставлено_за_выбранный_период', 'Заказано_на_сумму', 'Заказано_товаров_по_акции', 'Заказано_на_сумму_по_акции', 'Отправления_с_премиумподпиской_принятые_в_обработку']


<ipython-input-5-06ccb71f2ffc>:19: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(cols, axis=1, inplace=True)


Index(['Категория_3го_уровня', 'Товары', 'Модель', 'Ozon_ID', 'Артикул',
       'Акции', 'День', 'Уникальные_посетители_всего',
       'Уникальные_посетители_с_просмотром_карточки_товара', 'Показы_всего',
       'Показы_на_карточке_товара', 'Показы_в_поиске_и_каталоге',
       'Позиция_в_поиске_и_каталоге', 'В_корзину_всего', 'Заказано_товаров',
       'Доставлено_за_выбранный_период', 'Заказано_на_сумму',
       'Заказано_товаров_по_акции', 'Заказано_на_сумму_по_акции',
       'Отправления_с_премиумподпиской_принятые_в_обработку'],
      dtype='object')

In [ ]:
cols = ['Категория_3го_уровня','Товары', 'Модель', 'Артикул', 'Уникальные_посетители_всего',
       'Уникальные_посетители_с_просмотром_карточки_товара', 'Показы_всего',
       'Показы_на_карточке_товара', 'Показы_в_поиске_и_каталоге',
       'Позиция_в_поиске_и_каталоге', 'В_корзину_всего','Заказано_товаров_по_акции',
       'Доставлено_за_выбранный_период','Заказано_на_сумму_по_акции',
       'Отправления_с_премиумподпиской_принятые_в_обработку']
df.drop(cols, axis=1, inplace=True)
df.head(3)

,Ozon_ID,Акции,День,Заказано_товаров,Заказано_на_сумму
0,882353427,NaN,2023-03-05,0,0
1,882353427,NaN,2023-03-06,0,0
2,882353427,NaN,2023-03-07,0,0


In [ ]:
re_col = {'Заказано_товаров': 'Заказано_по_Акции','Заказано_на_сумму':'Заказано_на_сумму_по_акции'}
df.rename(columns=re_col, inplace=True)

In [ ]:
# Сборный индекс - для удобной группировки
get_index(df)
float2int(df)

In [ ]:
df['Ozon_ID'] = df['Ozon_ID'].astype(str)
# act['Артикул'] = act['Артикул'].astype(str)

df['День'] = pd.to_datetime(df['День'])
df['Акции'] = np.where(df.notna()['Акции'],1,0)

In [ ]:
df.dtypes

Ozon_ID                               object
Акции                                  int64
День                          datetime64[ns]
Заказано_по_Акции                      int64
Заказано_на_сумму_по_акции             int64
s_index                               object
dtype: object

In [ ]:
df.columns

Index(['Ozon_ID', 'Акции', 'День', 'Заказано_по_Акции',
       'Заказано_на_сумму_по_акции', 's_index'],
      dtype='object')

In [ ]:
all_df['Акции'] = df.copy() # до группировки

In [ ]:
#t = df.groupby(['s_index', 'День','Ozon_ID', 'Акции']).agg({'Заказано_товаров':'sum',
t = df.groupby('s_index').agg({'Акции':'max','Заказано_по_Акции':'sum',
                                'Заказано_на_сумму_по_акции':'sum'}).reset_index()

In [ ]:
t

,s_index,Акции,Заказано_по_Акции,Заказано_на_сумму_по_акции
0,2609120352022-06-03,1,0,0
1,2609120352022-07-03,1,0,0
2,2609120352022-07-08,1,0,0
3,2609120352022-07-09,1,0,0
4,2609120352022-07-25,1,0,0
...,...,...,...,...
239290,9660180192023-05-27,1,0,0
239291,9660180192023-05-28,1,0,0
239292,9660180192023-05-29,1,0,0
239293,9660180192023-05-30,1,1,1459


In [ ]:
t[t['Заказано_по_Акции']>0]

,s_index,Акции,Заказано_по_Акции,Заказано_на_сумму_по_акции
16,2609120352022-11-02,1,2,6080
74,2609196922022-08-21,1,2,9880
77,2609196922022-08-24,1,2,9880
107,2609196922022-09-27,1,2,9880
115,2609196922022-10-05,1,1,4940
...,...,...,...,...
239098,9315290582023-05-16,1,1,1640
239108,9315290582023-05-26,1,1,1640
239155,9323847992023-05-22,1,1,4900
239273,9660180192023-05-10,1,1,1640


In [ ]:
all_df['Акции_груп'] = t.copy() # с группировкой

###### Объединяем основные таблицы

In [ ]:
df_svod = pd.DataFrame()

In [ ]:
print(len(all_df['Графики']))
all_df['Графики'].columns

254069


Index(['s_index', 'Категория_3го_уровня', 'Ozon_ID', 'Артикул', 'День',
       'Посетители', 'Посетители_на_карточке_товара', 'Показы',
       'Позиция_в_поиске_и_каталоге', 'В_корзину', 'Заказано', 'Отменено',
       'Доставлено', 'Возвращено', 'Оборот_в_месяц_целевой'],
      dtype='object')

In [ ]:
print(len(all_df['Акции_груп']))
all_df['Акции_груп'].columns

239295


Index(['s_index', 'Акции', 'Заказано_по_Акции', 'Заказано_на_сумму_по_акции'], dtype='object')

In [ ]:
#df_svod = pd.merge(all_df['Графики'],all_df['Акции'], on=['День', 'Ozon_ID'], how='left')
df_svod = pd.merge(all_df['Графики'],all_df['Акции_груп'], on='s_index', how='left')

In [ ]:
df_svod.columns

Index(['s_index', 'Категория_3го_уровня', 'Ozon_ID', 'Артикул', 'День',
       'Посетители', 'Посетители_на_карточке_товара', 'Показы',
       'Позиция_в_поиске_и_каталоге', 'В_корзину', 'Заказано', 'Отменено',
       'Доставлено', 'Возвращено', 'Оборот_в_месяц_целевой', 'Акции',
       'Заказано_по_Акции', 'Заказано_на_сумму_по_акции'],
      dtype='object')

In [ ]:
df = df_svod.copy()

In [ ]:
df.columns

Index(['s_index', 'Категория_3го_уровня', 'Ozon_ID', 'Артикул', 'День',
       'Посетители', 'Посетители_на_карточке_товара', 'Показы',
       'Позиция_в_поиске_и_каталоге', 'В_корзину', 'Заказано', 'Отменено',
       'Доставлено', 'Возвращено', 'Оборот_в_месяц_целевой', 'Акции',
       'Заказано_по_Акции', 'Заказано_на_сумму_по_акции'],
      dtype='object')

Есть вопросы по колонкам из Акций:

'Заказано_по_акции',

'Заказано_на_сумму_y',

'Заказано_товаров_по_акции',

'Заказано_на_сумму_по_акции'

In [ ]:
df['Средний_чек'] = df['Оборот_в_месяц_целевой'] / df['Заказано']

In [ ]:
df.dtypes

s_index                                  object
Категория_3го_уровня                     object
Ozon_ID                                  object
Артикул                                  object
День                             datetime64[ns]
Посетители                                int64
Посетители_на_карточке_товара             int64
Показы                                    int64
Позиция_в_поиске_и_каталоге               int64
В_корзину                                 int64
Заказано                                  int64
Отменено                                  int64
Доставлено                                int64
Возвращено                                int64
Оборот_в_месяц_целевой                    int64
Акции                                   float64
Заказано_по_Акции                       float64
Заказано_на_сумму_по_акции              float64
Средний_чек                             float64
dtype: object

In [ ]:
float2int(df)

In [ ]:
df.isna().mean().sort_values(ascending=False)

s_index                          0.0
Заказано                         0.0
Заказано_на_сумму_по_акции       0.0
Заказано_по_Акции                0.0
Акции                            0.0
Оборот_в_месяц_целевой           0.0
Возвращено                       0.0
Доставлено                       0.0
Отменено                         0.0
В_корзину                        0.0
Категория_3го_уровня             0.0
Позиция_в_поиске_и_каталоге      0.0
Показы                           0.0
Посетители_на_карточке_товара    0.0
Посетители                       0.0
День                             0.0
Артикул                          0.0
Ozon_ID                          0.0
Средний_чек                      0.0
dtype: float64

In [ ]:
df.head(3)

,s_index,Категория_3го_уровня,Ozon_ID,Артикул,День,Посетители,Посетители_на_карточке_товара,Показы,Позиция_в_поиске_и_каталоге,В_корзину,Заказано,Отменено,Доставлено,Возвращено,Оборот_в_месяц_целевой,Акции,Заказано_по_Акции,Заказано_на_сумму_по_акции,Средний_чек
0,2609022712022-06-02,Лампа автомобильная,260902271,25541,2022-06-02,1,0,1,0,0,0,0,0,0,0,0,0,0,0
1,2609022712022-06-03,Лампа автомобильная,260902271,25541,2022-06-03,1,0,1,0,0,0,0,0,0,0,0,0,0,0
2,2609022712022-06-07,Лампа автомобильная,260902271,25541,2022-06-07,1,1,1,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
df.columns

Index(['s_index', 'Категория_3го_уровня', 'Ozon_ID', 'Артикул', 'День',
       'Посетители', 'Посетители_на_карточке_товара', 'Показы',
       'Позиция_в_поиске_и_каталоге', 'В_корзину', 'Заказано', 'Отменено',
       'Доставлено', 'Возвращено', 'Оборот_в_месяц_целевой', 'Акции',
       'Заказано_по_Акции', 'Заказано_на_сумму_по_акции', 'Средний_чек'],
      dtype='object')

In [ ]:
df.shape

(254069, 19)

In [ ]:
df.groupby(['s_index', 'Категория_3го_уровня', 'Ozon_ID', 'Артикул', 'День']).agg({'Посетители':'sum',
                                    'Посетители_на_карточке_товара':'sum',
                                    'Показы':'sum',
                                    'Позиция_в_поиске_и_каталоге':'min',
                                    'В_корзину':'sum', 'Заказано':'sum',
                                    'Отменено':'sum',
                                    'Доставлено':'sum',
                                    'Возвращено':'sum',
                                    'Оборот_в_месяц_целевой':'sum',
                                    'Акции':'max',
                                    'Заказано_по_Акции':'sum',
                                    'Заказано_на_сумму_по_акции':'sum',
                                    'Средний_чек':'mean'}).reset_index()

,s_index,Категория_3го_уровня,Ozon_ID,Артикул,День,Посетители,Посетители_на_карточке_товара,Показы,Позиция_в_поиске_и_каталоге,В_корзину,Заказано,Отменено,Доставлено,Возвращено,Оборот_в_месяц_целевой,Акции,Заказано_по_Акции,Заказано_на_сумму_по_акции,Средний_чек
0,2609022712022-06-02,Лампа автомобильная,260902271,25541,2022-06-02,1,0,1,0,0,0,0,0,0,0,0,0,0,0.0
1,2609022712022-06-03,Лампа автомобильная,260902271,25541,2022-06-03,1,0,1,0,0,0,0,0,0,0,0,0,0,0.0
2,2609022712022-06-07,Лампа автомобильная,260902271,25541,2022-06-07,1,1,1,0,0,0,0,0,0,0,0,0,0,0.0
3,2609022712022-06-08,Лампа автомобильная,260902271,25541,2022-06-08,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
4,2609022712022-06-12,Лампа автомобильная,260902271,25541,2022-06-12,1,0,1,0,0,0,0,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254064,9660743052023-05-28,Лампа автомобильная,966074305,34599,2023-05-28,19,0,19,132,0,0,0,0,0,0,0,0,0,0.0
254065,9660743052023-05-29,Лампа автомобильная,966074305,34599,2023-05-29,26,1,28,155,0,0,0,0,0,0,0,0,0,0.0
254066,9660743052023-05-30,Лампа автомобильная,966074305,34599,2023-05-30,36,3,43,59,0,0,0,0,0,0,0,0,0,0.0
254067,9660743052023-05-31,Лампа автомобильная,966074305,34599,2023-05-31,21,3,30,144,1,0,0,0,0,0,0,0,0,0.0


In [ ]:
df.shape

(254069, 19)

In [ ]:
mpoz = df['Позиция_в_поиске_и_каталоге'].max()
mpoz

4584

In [ ]:
df['Позиция_в_поиске_и_каталоге'] = np.where((df['Позиция_в_поиске_и_каталоге']==0),
                                             mpoz, df['Позиция_в_поиске_и_каталоге'])

In [ ]:
df[df['Позиция_в_поиске_и_каталоге'] == 0] # Проверяем

,s_index,Категория_3го_уровня,Ozon_ID,Артикул,День,Посетители,Посетители_на_карточке_товара,Показы,Позиция_в_поиске_и_каталоге,В_корзину,Заказано,Отменено,Доставлено,Возвращено,Оборот_в_месяц_целевой,Акции,Заказано_по_Акции,Заказано_на_сумму_по_акции,Средний_чек


In [ ]:
df.columns

Index(['s_index', 'Категория_3го_уровня', 'Ozon_ID', 'Артикул', 'День',
       'Посетители', 'Посетители_на_карточке_товара', 'Показы',
       'Позиция_в_поиске_и_каталоге', 'В_корзину', 'Заказано', 'Отменено',
       'Доставлено', 'Возвращено', 'Оборот_в_месяц_целевой', 'Акции',
       'Заказано_по_Акции', 'Заказано_на_сумму_по_акции', 'Средний_чек'],
      dtype='object')

In [ ]:
baza = all_df['База']
baza.head(3)

,Артикул,Ozon_ID,FBO_ID,FBS_ID,Контент_рейтинг,Текущая_цена
0,30273,105606938,298131117,298131122,15,1200
1,30383,105742204,298264198,298264234,38,1990
2,31284,112525698,303785665,303785701,30,7900


In [ ]:
len(baza['Артикул'].unique())

1259

In [ ]:
len(baza['Ozon_ID'].unique())

1259

In [ ]:
len(df['Ozon_ID'].unique())

1233

In [ ]:
len(df['Артикул'].unique())

994

In [ ]:
len(baza['FBS_ID'].unique())

1249

In [ ]:
baza.dtypes

Артикул            object
Ozon_ID            object
FBO_ID             object
FBS_ID             object
Контент_рейтинг     int64
Текущая_цена        int64
dtype: object

In [ ]:
df.dtypes

s_index                                  object
Категория_3го_уровня                     object
Ozon_ID                                  object
Артикул                                  object
День                             datetime64[ns]
Посетители                                int64
Посетители_на_карточке_товара             int64
Показы                                    int64
Позиция_в_поиске_и_каталоге               int64
В_корзину                                 int64
Заказано                                  int64
Отменено                                  int64
Доставлено                                int64
Возвращено                                int64
Оборот_в_месяц_целевой                    int64
Акции                                     int64
Заказано_по_Акции                         int64
Заказано_на_сумму_по_акции                int64
Средний_чек                               int64
dtype: object

In [ ]:
# uniq_b = np.intersect1d(baza['FBS_ID'].unique(), (df['Ozon_ID'].unique()))
# w = uniq_b.tolist()

# print(f'Уникальных в Базе: {}')
# print(f'Уникальных в Своде: {len(df['Ozon_ID'].unique())}')
# print(f'Общих: {len(w)}')

In [ ]:
baza['FBS_ID'].unique()

array(['298131122', '298264234', '303785701', ..., '278434864',
       '278437421', '283791079'], dtype=object)

In [ ]:
len(baza['FBS_ID'].unique())

1249

In [ ]:
b = baza[['FBS_ID','Контент_рейтинг','Текущая_цена']].copy()

In [ ]:
b.rename(columns={'FBS_ID':'Ozon_ID'}, inplace=True)

In [ ]:
b.head(3)

,Ozon_ID,Контент_рейтинг,Текущая_цена
0,298131122,15,1200
1,298264234,38,1990
2,303785701,30,7900


In [ ]:
df.head(3)

,s_index,Категория_3го_уровня,Ozon_ID,Артикул,День,Посетители,Посетители_на_карточке_товара,Показы,Позиция_в_поиске_и_каталоге,В_корзину,Заказано,Отменено,Доставлено,Возвращено,Оборот_в_месяц_целевой,Акции,Заказано_по_Акции,Заказано_на_сумму_по_акции,Средний_чек
0,2609022712022-06-02,Лампа автомобильная,260902271,25541,2022-06-02,1,0,1,4584,0,0,0,0,0,0,0,0,0,0
1,2609022712022-06-03,Лампа автомобильная,260902271,25541,2022-06-03,1,0,1,4584,0,0,0,0,0,0,0,0,0,0
2,2609022712022-06-07,Лампа автомобильная,260902271,25541,2022-06-07,1,1,1,4584,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_svod = pd.merge(df,b, on='Ozon_ID', how='left')

In [ ]:
float2int(df_svod)
df_svod.head(3)

,s_index,Категория_3го_уровня,Ozon_ID,Артикул,День,Посетители,Посетители_на_карточке_товара,Показы,Позиция_в_поиске_и_каталоге,В_корзину,...,Отменено,Доставлено,Возвращено,Оборот_в_месяц_целевой,Акции,Заказано_по_Акции,Заказано_на_сумму_по_акции,Средний_чек,Контент_рейтинг,Текущая_цена
0,2609022712022-06-02,Лампа автомобильная,260902271,25541,2022-06-02,1,0,1,4584,0,...,0,0,0,0,0,0,0,0,0,0
1,2609022712022-06-03,Лампа автомобильная,260902271,25541,2022-06-03,1,0,1,4584,0,...,0,0,0,0,0,0,0,0,0,0
2,2609022712022-06-07,Лампа автомобильная,260902271,25541,2022-06-07,1,1,1,4584,0,...,0,0,0,0,0,0,0,0,0,0


###### Ozon/Performance/Трафареты

In [ ]:
all_traf = pd.DataFrame()

In [ ]:
col = ['День', 'sku', 'Показы', 'Клики',
       'Расход, ₽, с НДС', 'Заказы',
       'Выручка, ₽', 'Заказы модели', 'Выручка с заказов модели, ₽']
for i in traf_paths:
  df = pd.read_csv(traf_paths[i], skiprows=[0], skipfooter=1, delimiter=';',decimal=',', usecols=col) #, skiprows=[0,2], usecols=['column1', 'column2']
  print('*'*40)
  print(f'df - {i}')
  print(df.shape)
  print(df.columns)
  normal_col(df)
  df.dropna(subset=['sku'], inplace=True) # удаляем строки с пустым ID
  # try:
  #   cols = ['Название_товара', 'Цена_товара',
  #         'CTR_%', 'Ср_цена_1000_показов']
  #   df.drop(cols, axis=1, inplace=True)
  # except:
  #   cols = ['Название_товара', 'Цена_товара','CTR_%','Ср_цена_клика']
  #   df.drop(cols, axis=1, inplace=True)

  df['sku'] = df['sku'].astype(int)
  df.rename(columns={'sku': 'Ozon_ID'}, inplace=True)
  df['День'] = pd.to_datetime(df['День'])
  df['Ozon_ID'] = df['Ozon_ID'].astype(str)
  # df['Расход__с_НДС'] = df['Расход__с_НДС'].replace(',','.', regex=True)
  df['Расход__с_НДС'] = df['Расход__с_НДС'].astype(float)
  # df['Выручка'] = df['Выручка'].replace(',','.', regex=True)
  df['Выручка'] = df['Выручка'].astype(float)
  # df['Выручка_с_заказов_модели'] = df['Выручка_с_заказов_модели'].replace(',','.', regex=True)
  df['Выручка_с_заказов_модели'] = df['Выручка_с_заказов_модели'].astype(float)
  if df.shape[0] != 1:
    all_traf = pd.concat([all_traf, df], ignore_index=True)


<ipython-input-21-f78cc1312b88>:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(traf_paths[i], skiprows=[0], skipfooter=1, delimiter=';',decimal=',', usecols=col) #, skiprows=[0,2], usecols=['column1', 'column2']
<ipython-input-15-06ccb71f2ffc>:19: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(cols, axis=1, inplace=True)
<ipython-input-21-f78cc1312b88>:22: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['День'] = pd.to_datetime(df['День'])
<ipython-input-21-f78cc1312b88>:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avo

****************************************
df - 3263861
(254, 9)
Index(['День', 'sku', 'Показы', 'Клики', 'Расход, ₽, с НДС', 'Заказы',
       'Выручка, ₽', 'Заказы модели', 'Выручка с заказов модели, ₽'],
      dtype='object')
['День', 'sku', 'Показы', 'Клики', 'Расход__с_НДС', 'Заказы', 'Выручка', 'Заказы_модели', 'Выручка_с_заказов_модели']
****************************************
df - 2983852
(110, 9)
Index(['День', 'sku', 'Показы', 'Клики', 'Расход, ₽, с НДС', 'Заказы',
       'Выручка, ₽', 'Заказы модели', 'Выручка с заказов модели, ₽'],
      dtype='object')
['День', 'sku', 'Показы', 'Клики', 'Расход__с_НДС', 'Заказы', 'Выручка', 'Заказы_модели', 'Выручка_с_заказов_модели']
****************************************
df - 3225021
(257, 9)
Index(['День', 'sku', 'Показы', 'Клики', 'Расход, ₽, с НДС', 'Заказы',
       'Выручка, ₽', 'Заказы модели', 'Выручка с заказов модели, ₽'],
      dtype='object')
['День', 'sku', 'Показы', 'Клики', 'Расход__с_НДС', 'Заказы', 'Выручка', 'Заказы_модел

<ipython-input-21-f78cc1312b88>:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(traf_paths[i], skiprows=[0], skipfooter=1, delimiter=';',decimal=',', usecols=col) #, skiprows=[0,2], usecols=['column1', 'column2']
<ipython-input-15-06ccb71f2ffc>:19: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(cols, axis=1, inplace=True)
<ipython-input-21-f78cc1312b88>:22: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['День'] = pd.to_datetime(df['День'])
<ipython-input-21-f78cc1312b88>:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avo

****************************************
df - 2651789_01.05.2023-31.05.2023
(447, 9)
Index(['День', 'sku', 'Показы', 'Клики', 'Расход, ₽, с НДС', 'Заказы',
       'Выручка, ₽', 'Заказы модели', 'Выручка с заказов модели, ₽'],
      dtype='object')
['День', 'sku', 'Показы', 'Клики', 'Расход__с_НДС', 'Заказы', 'Выручка', 'Заказы_модели', 'Выручка_с_заказов_модели']
****************************************
df - 2651789_07.03.2023-30.04.2023
(1077, 9)
Index(['День', 'sku', 'Показы', 'Клики', 'Расход, ₽, с НДС', 'Заказы',
       'Выручка, ₽', 'Заказы модели', 'Выручка с заказов модели, ₽'],
      dtype='object')
['День', 'sku', 'Показы', 'Клики', 'Расход__с_НДС', 'Заказы', 'Выручка', 'Заказы_модели', 'Выручка_с_заказов_модели']
****************************************
df - 1615846_24.11.2022-24.01.2023
(1425, 9)
Index(['День', 'sku', 'Показы', 'Клики', 'Расход, ₽, с НДС', 'Заказы',
       'Выручка, ₽', 'Заказы модели', 'Выручка с заказов модели, ₽'],
      dtype='object')
['День', 'sku', 'П

<ipython-input-15-06ccb71f2ffc>:19: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(cols, axis=1, inplace=True)
<ipython-input-21-f78cc1312b88>:22: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['День'] = pd.to_datetime(df['День'])
<ipython-input-21-f78cc1312b88>:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(traf_paths[i], skiprows=[0], skipfooter=1, delimiter=';',decimal=',', usecols=col) #, skiprows=[0,2], usecols=['column1', 'column2']
<ipython-input-15-06ccb71f2ffc>:19: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `o

****************************************
df - 1223100_23.08.2022-23.10.2022
(12014, 9)
Index(['День', 'sku', 'Показы', 'Клики', 'Расход, ₽, с НДС', 'Заказы',
       'Выручка, ₽', 'Заказы модели', 'Выручка с заказов модели, ₽'],
      dtype='object')
['День', 'sku', 'Показы', 'Клики', 'Расход__с_НДС', 'Заказы', 'Выручка', 'Заказы_модели', 'Выручка_с_заказов_модели']
****************************************
df - 1223100_26.02.2023-26.04.2023
(0, 9)
Index(['День', 'sku', 'Показы', 'Клики', 'Расход, ₽, с НДС', 'Заказы',
       'Выручка, ₽', 'Заказы модели', 'Выручка с заказов модели, ₽'],
      dtype='object')
['День', 'sku', 'Показы', 'Клики', 'Расход__с_НДС', 'Заказы', 'Выручка', 'Заказы_модели', 'Выручка_с_заказов_модели']
****************************************
df - 1223100_27.04.2023-31.05.2023
(0, 9)
Index(['День', 'sku', 'Показы', 'Клики', 'Расход, ₽, с НДС', 'Заказы',
       'Выручка, ₽', 'Заказы модели', 'Выручка с заказов модели, ₽'],
      dtype='object')
['День', 'sku', 'Показ

<ipython-input-15-06ccb71f2ffc>:19: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(cols, axis=1, inplace=True)
<ipython-input-21-f78cc1312b88>:22: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['День'] = pd.to_datetime(df['День'])
<ipython-input-21-f78cc1312b88>:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(traf_paths[i], skiprows=[0], skipfooter=1, delimiter=';',decimal=',', usecols=col) #, skiprows=[0,2], usecols=['column1', 'column2']
<ipython-input-15-06ccb71f2ffc>:19: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `o

****************************************
df - 1223100_22.06.2022-22.08.2022
(11257, 9)
Index(['День', 'sku', 'Показы', 'Клики', 'Расход, ₽, с НДС', 'Заказы',
       'Выручка, ₽', 'Заказы модели', 'Выручка с заказов модели, ₽'],
      dtype='object')
['День', 'sku', 'Показы', 'Клики', 'Расход__с_НДС', 'Заказы', 'Выручка', 'Заказы_модели', 'Выручка_с_заказов_модели']
****************************************
df - 2651630_07.03.2023-07.03.2023
(25, 9)
Index(['День', 'sku', 'Показы', 'Клики', 'Расход, ₽, с НДС', 'Заказы',
       'Выручка, ₽', 'Заказы модели', 'Выручка с заказов модели, ₽'],
      dtype='object')
['День', 'sku', 'Показы', 'Клики', 'Расход__с_НДС', 'Заказы', 'Выручка', 'Заказы_модели', 'Выручка_с_заказов_модели']
****************************************
df - 2651630_01.05.2023-31.05.2023
(418, 9)
Index(['День', 'sku', 'Показы', 'Клики', 'Расход, ₽, с НДС', 'Заказы',
       'Выручка, ₽', 'Заказы модели', 'Выручка с заказов модели, ₽'],
      dtype='object')
['День', 'sku', 'По

<ipython-input-15-06ccb71f2ffc>:19: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(cols, axis=1, inplace=True)
<ipython-input-21-f78cc1312b88>:22: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['День'] = pd.to_datetime(df['День'])
<ipython-input-21-f78cc1312b88>:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(traf_paths[i], skiprows=[0], skipfooter=1, delimiter=';',decimal=',', usecols=col) #, skiprows=[0,2], usecols=['column1', 'column2']
<ipython-input-15-06ccb71f2ffc>:19: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `o

In [ ]:
#all_traf.drop(['Расход_за_минусом_бонусов__с_НДС'], axis=1, inplace=True)

In [ ]:
all_traf['Трафареты_Количество_заказов'] = all_traf['Заказы']+all_traf['Заказы_модели']
all_traf['Трафареты_Сумма_заказов'] = all_traf['Выручка']+all_traf['Выручка_с_заказов_модели']

In [ ]:
col = {'Показы':'Трафареты_Охват',
       'Клики':'Трафареты_Клики',
       'Расход__с_НДС':'Трафареты_Расходы_всего'}
all_traf.rename(columns=col, inplace=True)
all_traf.drop(['Заказы','Выручка', 'Заказы_модели', 'Выручка_с_заказов_модели'], axis=1, inplace=True)

In [ ]:
all_traf.columns

Index(['День', 'Ozon_ID', 'Трафареты_Охват', 'Трафареты_Клики',
       'Трафареты_Расходы_всего', 'Трафареты_Количество_заказов',
       'Трафареты_Сумма_заказов'],
      dtype='object')

In [ ]:
df = all_traf.copy()

In [ ]:
df['Ozon_ID'] = df['Ozon_ID'].astype(str)
df.dtypes

День                            datetime64[ns]
Ozon_ID                                 object
Трафареты_Охват                          int64
Трафареты_Клики                          int64
Трафареты_Расходы_всего                  int64
Трафареты_Количество_заказов             int64
Трафареты_Сумма_заказов                  int64
dtype: object

In [ ]:
float2int(df)

In [ ]:
get_index(df)

In [ ]:
df.drop(['День','Ozon_ID'], axis=1, inplace=True)

In [ ]:
df

,Трафареты_Охват,Трафареты_Клики,Трафареты_Расходы_всего,Трафареты_Количество_заказов,Трафареты_Сумма_заказов
0,379,3,4,0,0
1,287,4,3,0,0
2,226,6,2,0,0
3,685,20,7,0,0
4,222,6,2,0,0
...,...,...,...,...,...
56242,49,1,1,0,0
56243,85,3,3,0,0
56244,161,3,5,0,0
56245,1361,22,41,2,6728


In [ ]:
all_df['Трафареты'] = df.copy()

Добавляем к Сводному ДФ

In [ ]:
df_svod = pd.merge(df_svod,df, on='s_index', how='left')

In [ ]:
df_svod

,s_index,Категория_3го_уровня,Ozon_ID,Артикул,День,Посетители,Посетители_на_карточке_товара,Показы,Позиция_в_поиске_и_каталоге,В_корзину,...,Заказано_по_Акции,Заказано_на_сумму_по_акции,Средний_чек,Контент_рейтинг,Текущая_цена,Трафареты_Охват,Трафареты_Клики,Трафареты_Расходы_всего,Трафареты_Количество_заказов,Трафареты_Сумма_заказов
0,2609022712022-06-02,Лампа автомобильная,260902271,25541,2022-06-02,1,0,1,4584,0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
1,2609022712022-06-03,Лампа автомобильная,260902271,25541,2022-06-03,1,0,1,4584,0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
2,2609022712022-06-07,Лампа автомобильная,260902271,25541,2022-06-07,1,1,1,4584,0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
3,2609022712022-06-08,Лампа автомобильная,260902271,25541,2022-06-08,0,0,0,4584,0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
4,2609022712022-06-12,Лампа автомобильная,260902271,25541,2022-06-12,1,0,1,4584,0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254466,9660743052023-05-28,Лампа автомобильная,966074305,34599,2023-05-28,19,0,19,132,0,...,0,0,0,40,1640,NaN,NaN,NaN,NaN,NaN
254467,9660743052023-05-29,Лампа автомобильная,966074305,34599,2023-05-29,26,1,28,155,0,...,0,0,0,40,1640,NaN,NaN,NaN,NaN,NaN
254468,9660743052023-05-30,Лампа автомобильная,966074305,34599,2023-05-30,36,3,43,59,0,...,0,0,0,40,1640,NaN,NaN,NaN,NaN,NaN
254469,9660743052023-05-31,Лампа автомобильная,966074305,34599,2023-05-31,21,3,30,144,1,...,0,0,0,40,1640,NaN,NaN,NaN,NaN,NaN


###### Ozon/Performance/ПВП

In [ ]:
all_pvp = pd.DataFrame()

In [ ]:
col = ['Дата', 'Ozon ID','Артикул','Количество',
       'Цена продажи', 'Стоимость, ₽', 'Ставка, %', 'Ставка, ₽',
       'Расход, ₽']
for i in pvp_paths:
  df = pd.read_csv(pvp_paths['1415722_08.08.2022-08.10.2022'], skiprows=[0], skipfooter=1, sep=';', decimal=',', usecols=col, engine='python')
  #, skiprows=[0,2], usecols=['column1', 'column2']
  print('*'*40)
  print(f'df - {i}')
  print(df.shape)
  print(df.columns)
  normal_col(df)
  df.dropna(subset=['Ozon_ID'], inplace=True) # удаляем строки с пустым ID
  # try:
  #   cols = ['Номер_заказа', 'ID_заказа','Ozon_ID_рекламируемого_товара',
  #           'Наименование','Расход']
  #   df.drop(cols, axis=1, inplace=True)
  # except:
  #   # cols = ['Название_товара', 'Цена_товара','CTR','Ср_цена_клика']
  #   # df.drop(cols, axis=1, inplace=True)
  #   print('Шо-то тут не то')
  df['Ozon_ID'] = df['Ozon_ID'].astype(int)
  df['Дата'] = pd.to_datetime(df['Дата'])
  df.rename(columns={'Дата': 'День'}, inplace=True)
  df['Ozon_ID'] = df['Ozon_ID'].astype(str)
  # df['Ставка'] = df['Ставка'].replace(',','.', regex=True)
  df['Ставка'] = df['Ставка'].astype(float)
  # df['Ставка_%'] = df['Ставка_%'].replace(',','.', regex=True)
  df['Ставка_%'] = df['Ставка_%'].astype(float)
  # df['Стоимость'] = df['Стоимость'].replace(',','.', regex=True)
  df['Стоимость'] = df['Стоимость'].astype(float)
  # df['Цена_продажи'] = df['Цена_продажи'].replace(',','.', regex=True)
  df['Цена_продажи'] = df['Цена_продажи'].astype(float)
  df['Количество'] = df['Количество'].astype(int)
  if df.shape[0] > 1:
    all_pvp = pd.concat([all_pvp, df], ignore_index=True)
  print(df.dtypes)

****************************************
df - 3040816
(353, 9)
Index(['Дата', 'Ozon ID', 'Артикул', 'Количество', 'Цена продажи',
       'Стоимость, ₽', 'Ставка, %', 'Ставка, ₽', 'Расход, ₽'],
      dtype='object')
['Дата', 'Ozon_ID', 'Артикул', 'Количество', 'Цена_продажи', 'Стоимость', 'Ставка_%', 'Ставка', 'Расход']
День            datetime64[ns]
Ozon_ID                 object
Артикул                  int64
Количество               int64
Цена_продажи           float64
Стоимость              float64
Ставка_%               float64
Ставка                 float64
Расход                 float64
dtype: object
****************************************
df - 1415722_11.02.2023-11.04.2023
(353, 9)
Index(['Дата', 'Ozon ID', 'Артикул', 'Количество', 'Цена продажи',
       'Стоимость, ₽', 'Ставка, %', 'Ставка, ₽', 'Расход, ₽'],
      dtype='object')
['Дата', 'Ozon_ID', 'Артикул', 'Количество', 'Цена_продажи', 'Стоимость', 'Ставка_%', 'Ставка', 'Расход']
День            datetime64[ns]
Ozon_ID     

<ipython-input-15-06ccb71f2ffc>:19: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(cols, axis=1, inplace=True)
<ipython-input-58-9f52a57ac7e5>:22: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['Дата'] = pd.to_datetime(df['Дата'])
<ipython-input-15-06ccb71f2ffc>:19: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(cols, axis=1, inplace=True)
<ipython-input-58-9f52a57ac7e5>:22: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['Дата'] = pd.to_datetime(df['Да

День            datetime64[ns]
Ozon_ID                 object
Артикул                  int64
Количество               int64
Цена_продажи           float64
Стоимость              float64
Ставка_%               float64
Ставка                 float64
Расход                 float64
dtype: object
****************************************
df - 1613046_22.09.2022-22.11.2022
(353, 9)
Index(['Дата', 'Ozon ID', 'Артикул', 'Количество', 'Цена продажи',
       'Стоимость, ₽', 'Ставка, %', 'Ставка, ₽', 'Расход, ₽'],
      dtype='object')
['Дата', 'Ozon_ID', 'Артикул', 'Количество', 'Цена_продажи', 'Стоимость', 'Ставка_%', 'Ставка', 'Расход']
День            datetime64[ns]
Ozon_ID                 object
Артикул                  int64
Количество               int64
Цена_продажи           float64
Стоимость              float64
Ставка_%               float64
Ставка                 float64
Расход                 float64
dtype: object
****************************************
df - 1613046_26.05.2023-31.05.

<ipython-input-15-06ccb71f2ffc>:19: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(cols, axis=1, inplace=True)
<ipython-input-58-9f52a57ac7e5>:22: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['Дата'] = pd.to_datetime(df['Дата'])
<ipython-input-15-06ccb71f2ffc>:19: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(cols, axis=1, inplace=True)
<ipython-input-58-9f52a57ac7e5>:22: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['Дата'] = pd.to_datetime(df['Да

****************************************
df - 1612196_23.11.2022-23.01.2023
(353, 9)
Index(['Дата', 'Ozon ID', 'Артикул', 'Количество', 'Цена продажи',
       'Стоимость, ₽', 'Ставка, %', 'Ставка, ₽', 'Расход, ₽'],
      dtype='object')
['Дата', 'Ozon_ID', 'Артикул', 'Количество', 'Цена_продажи', 'Стоимость', 'Ставка_%', 'Ставка', 'Расход']
День            datetime64[ns]
Ozon_ID                 object
Артикул                  int64
Количество               int64
Цена_продажи           float64
Стоимость              float64
Ставка_%               float64
Ставка                 float64
Расход                 float64
dtype: object
****************************************
df - 1612196_26.05.2023-31.05.2023
(353, 9)
Index(['Дата', 'Ozon ID', 'Артикул', 'Количество', 'Цена продажи',
       'Стоимость, ₽', 'Ставка, %', 'Ставка, ₽', 'Расход, ₽'],
      dtype='object')
['Дата', 'Ozon_ID', 'Артикул', 'Количество', 'Цена_продажи', 'Стоимость', 'Ставка_%', 'Ставка', 'Расход']
День            datet

<ipython-input-15-06ccb71f2ffc>:19: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(cols, axis=1, inplace=True)
<ipython-input-58-9f52a57ac7e5>:22: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['Дата'] = pd.to_datetime(df['Дата'])
<ipython-input-15-06ccb71f2ffc>:19: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(cols, axis=1, inplace=True)
<ipython-input-58-9f52a57ac7e5>:22: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['Дата'] = pd.to_datetime(df['Да

****************************************
df - 1606841_21.09.2022-21.11.2022
(353, 9)
Index(['Дата', 'Ozon ID', 'Артикул', 'Количество', 'Цена продажи',
       'Стоимость, ₽', 'Ставка, %', 'Ставка, ₽', 'Расход, ₽'],
      dtype='object')
['Дата', 'Ozon_ID', 'Артикул', 'Количество', 'Цена_продажи', 'Стоимость', 'Ставка_%', 'Ставка', 'Расход']
День            datetime64[ns]
Ozon_ID                 object
Артикул                  int64
Количество               int64
Цена_продажи           float64
Стоимость              float64
Ставка_%               float64
Ставка                 float64
Расход                 float64
dtype: object
****************************************
df - 1606841_23.01.2023-23.03.2023
(353, 9)
Index(['Дата', 'Ozon ID', 'Артикул', 'Количество', 'Цена продажи',
       'Стоимость, ₽', 'Ставка, %', 'Ставка, ₽', 'Расход, ₽'],
      dtype='object')
['Дата', 'Ozon_ID', 'Артикул', 'Количество', 'Цена_продажи', 'Стоимость', 'Ставка_%', 'Ставка', 'Расход']
День            datet

<ipython-input-58-9f52a57ac7e5>:22: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['Дата'] = pd.to_datetime(df['Дата'])
<ipython-input-15-06ccb71f2ffc>:19: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(cols, axis=1, inplace=True)
<ipython-input-58-9f52a57ac7e5>:22: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['Дата'] = pd.to_datetime(df['Дата'])
<ipython-input-15-06ccb71f2ffc>:19: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(cols, axis=1, inplace=

In [ ]:
df = all_pvp.copy()
df.shape

(8825, 9)

In [ ]:
float2int(df)

In [ ]:
df.head()

,День,Ozon_ID,Артикул,Количество,Цена_продажи,Стоимость,Ставка_%,Ставка,Расход
0,2022-08-08,298264234,30383,1,2190,2190,0,11,11
1,2022-09-08,511581300,29397,1,3000,3000,0,15,15
2,2022-09-08,510826421,31319,1,8990,8990,1,108,108
3,2022-09-08,556497014,33750,1,3980,3980,1,44,44
4,2022-09-08,511572338,29396,1,3000,3000,0,15,15


In [ ]:
col = {'Количество':'Продвижение_в_поиске_Количество_заказов',
       'Стоимость':'Продвижение_в_поиске_Сумма_заказов',
       'Ставка':'Продвижение_в_поиске_Ставка'}
df.rename(columns=col, inplace=True)

In [ ]:
get_index(df)

In [ ]:
# Меняем местами колонки
df = df.reindex(columns=['s_index','День', 'Ozon_ID', 'Артикул', 'Продвижение_в_поиске_Количество_заказов',
       'Цена_продажи', 'Продвижение_в_поиске_Сумма_заказов', 'Ставка_%',
       'Продвижение_в_поиске_Ставка'])

In [ ]:
df.columns

Index(['s_index', 'День', 'Ozon_ID', 'Артикул',
       'Продвижение_в_поиске_Количество_заказов', 'Цена_продажи',
       'Продвижение_в_поиске_Сумма_заказов', 'Ставка_%',
       'Продвижение_в_поиске_Ставка'],
      dtype='object')

In [ ]:
uniq_b = (df['Ozon_ID'].unique())
uniq_s = (df_svod['Ozon_ID'].unique())
w = []
for i in uniq_b:
  for j in uniq_s:
    if i == j:
      w.append(j)

print(f'Уникальных в БазеПВП: {len(uniq_b)}')
print(f'Уникальных в Своде: {len(uniq_s)}')
print(f'Общих: {len(w)}')

In [ ]:
col = ['День','Ozon_ID', 'Артикул']
df.drop(columns = col, inplace=True)

In [ ]:
df.head(3)

,Продвижение_в_поиске_Количество_заказов,Цена_продажи,Продвижение_в_поиске_Сумма_заказов,Ставка_%,Продвижение_в_поиске_Ставка,s_index
0,1,1590,1590,8,127,9307891872023-04-21
1,1,1640,1640,8,131,9315290582023-05-16
2,1,2190,2190,0,11,2982642342022-08-08


In [ ]:
all_df['ПВП'] = df.copy()

In [ ]:
df_svod = pd.merge(df_svod,df, on='s_index', how='left')

In [ ]:
float2int(df_svod)

In [ ]:
df_svod.head(4)

,s_index,Категория_3го_уровня,Ozon_ID,Артикул,День,Посетители,Посетители_на_карточке_товара,Показы,Позиция_в_поиске_и_каталоге,В_корзину,...,Трафареты_Охват,Трафареты_Клики,Трафареты_Расходы_всего,Трафареты_Количество_заказов,Трафареты_Сумма_заказов,Продвижение_в_поиске_Количество_заказов,Цена_продажи,Продвижение_в_поиске_Сумма_заказов,Ставка_%,Продвижение_в_поиске_Ставка
0,2609022712022-06-02,Лампа автомобильная,260902271,25541,2022-06-02,1,0,1,4584,0,...,0,0,0,0,0,0,0,0,0,0
1,2609022712022-06-03,Лампа автомобильная,260902271,25541,2022-06-03,1,0,1,4584,0,...,0,0,0,0,0,0,0,0,0,0
2,2609022712022-06-07,Лампа автомобильная,260902271,25541,2022-06-07,1,1,1,4584,0,...,0,0,0,0,0,0,0,0,0,0
3,2609022712022-06-08,Лампа автомобильная,260902271,25541,2022-06-08,0,0,0,4584,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_svod.dtypes

s_index                                            object
Категория_3го_уровня                               object
Ozon_ID                                            object
Артикул                                            object
День                                       datetime64[ns]
Посетители                                          int64
Посетители_на_карточке_товара                       int64
Показы                                              int64
Позиция_в_поиске_и_каталоге                         int64
В_корзину                                           int64
Заказано                                            int64
Отменено                                            int64
Доставлено                                          int64
Возвращено                                          int64
Оборот_в_месяц_целевой                              int64
Акции                                               int64
Заказано_по_Акции                                   int64
Заказано_на_су

###### 'Ozon/Товары по категориям'

In [ ]:
all_cat = pd.DataFrame()
for i in cat_paths:
  df = pd.read_excel(cat_paths[i],
                   sheet_name='Шаблон для поставщика', skiprows=[0,2],
                   usecols=['Артикул*', 'Ссылки на дополнительные фото'])
  print('*'*40)
  float2int(df)
  normal_col(df)
  print(f'df - {i}')
  print(df.shape)
  print(df.columns)

  if df.shape[0] > 1:
    all_cat = pd.concat([all_cat, df], ignore_index=True)
  print(df.dtypes)

****************************************
['Артикул', 'Ссылки_на_дополнительные_фото']
df - 2023-06-05 Автоаксессуар
(1, 2)
Index(['Артикул', 'Ссылки_на_дополнительные_фото'], dtype='object')
Артикул                           int64
Ссылки_на_дополнительные_фото    object
dtype: object
****************************************
['Артикул', 'Ссылки_на_дополнительные_фото']
df - 2023-06-05 Автомобильные лампы
(118, 2)
Index(['Артикул', 'Ссылки_на_дополнительные_фото'], dtype='object')
Артикул                           int64
Ссылки_на_дополнительные_фото    object
dtype: object


<ipython-input-5-06ccb71f2ffc>:19: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(cols, axis=1, inplace=True)
<ipython-input-5-06ccb71f2ffc>:19: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(cols, axis=1, inplace=True)


****************************************
['Артикул', 'Ссылки_на_дополнительные_фото']
df - 2023-06-05 Автомобильные лампы ксенон комплектующие
(576, 2)
Index(['Артикул', 'Ссылки_на_дополнительные_фото'], dtype='object')
Артикул                           int64
Ссылки_на_дополнительные_фото    object
dtype: object


<ipython-input-5-06ccb71f2ffc>:19: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(cols, axis=1, inplace=True)


****************************************
['Артикул', 'Ссылки_на_дополнительные_фото']
df - 2023-06-05 Автомобильные фары
(560, 2)
Index(['Артикул', 'Ссылки_на_дополнительные_фото'], dtype='object')
Артикул                           int64
Ссылки_на_дополнительные_фото    object
dtype: object
****************************************
['Артикул', 'Ссылки_на_дополнительные_фото']
df - 2023-06-05 Герметик
(4, 2)
Index(['Артикул', 'Ссылки_на_дополнительные_фото'], dtype='object')
Артикул                           int64
Ссылки_на_дополнительные_фото    object
dtype: object


<ipython-input-5-06ccb71f2ffc>:19: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(cols, axis=1, inplace=True)
<ipython-input-5-06ccb71f2ffc>:19: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(cols, axis=1, inplace=True)


In [ ]:
# # Тренируюсь на одной таблице
# df = pd.read_excel(cat_paths['2023-06-05 Автомобильные лампы ксенон комплектующие'],
#                    sheet_name='Шаблон для поставщика', skiprows=[0,2],
#                    usecols=['Артикул*', 'Ссылки на дополнительные фото']) #, skiprows=[0,2], usecols=['column1', 'column2']
# float2int(df)
# normal_col(df)
# df.head(3)

In [ ]:
df = all_cat.copy()

In [ ]:
# вычислить сколько доп.фото в колонке 'Ссылки на дополнительные фото' и прибавить 1
def get_col_foto1(x):
  # print(type(x), x)
  if x == 0:
    return 1
  else:
    dop = x.count('\n') + 1
    # print(dop)
    # print(f'Доп фото: {dop}')
  return 1 + dop

In [ ]:
df['Контент_Количество_фотографий'] = df['Ссылки_на_дополнительные_фото'].apply(get_col_foto1)
df['Артикул'] = df['Артикул'].astype(str)
df.drop('Ссылки_на_дополнительные_фото', axis=1, inplace=True)

In [ ]:
df.head(3)

,Артикул,Контент_Количество_фотографий
0,26223,1
1,28814,1
2,26312,2


In [ ]:
df.dtypes

Артикул                          object
Контент_Количество_фотографий     int64
dtype: object

In [ ]:
df['Артикул'] = df['Артикул'].astype(str)

In [ ]:
uniq_b = (df['Артикул'].unique())
uniq_s = (df_svod['Артикул'].unique())
w = []
for i in uniq_b:
  if i in uniq_s:
    w.append(j)

print(f'Уникальных в all_cat: {len(uniq_b)}')
print(f'Уникальных в Своде: {len(uniq_s)}')
print(f'Общих: {len(w)}')

Уникальных в all_cat: 1258
Уникальных в Своде: 994
Общих: 952


In [ ]:
float2int(df_svod)

In [ ]:
# Дополняем сводную таблицу
df_svod = pd.merge(df_svod,df, on='Артикул', how='left')


In [ ]:
df_svod.head()

,s_index,Категория_3го_уровня,Ozon_ID,Артикул,День,Посетители,Посетители_на_карточке_товара,Показы,Позиция_в_поиске_и_каталоге,В_корзину,...,Трафареты_Клики,Трафареты_Расходы_всего,Трафареты_Количество_заказов,Трафареты_Сумма_заказов,Продвижение_в_поиске_Количество_заказов,Цена_продажи,Продвижение_в_поиске_Сумма_заказов,Ставка_%,Продвижение_в_поиске_Ставка,Контент_Количество_фотографий
0,2609022712022-06-02,Лампа автомобильная,260902271,25541,2022-06-02,1,0,1,4584,0,...,0,0,0,0,0,0,0,0,0,NaN
1,2609022712022-06-03,Лампа автомобильная,260902271,25541,2022-06-03,1,0,1,4584,0,...,0,0,0,0,0,0,0,0,0,NaN
2,2609022712022-06-07,Лампа автомобильная,260902271,25541,2022-06-07,1,1,1,4584,0,...,0,0,0,0,0,0,0,0,0,NaN
3,2609022712022-06-08,Лампа автомобильная,260902271,25541,2022-06-08,0,0,0,4584,0,...,0,0,0,0,0,0,0,0,0,NaN
4,2609022712022-06-12,Лампа автомобильная,260902271,25541,2022-06-12,1,0,1,4584,0,...,0,0,0,0,0,0,0,0,0,NaN


In [ ]:
float2int(df_svod)

In [ ]:
df_svod[df_svod['Контент_Количество_фотографий']>1].head(5) #.isna()

,s_index,Категория_3го_уровня,Ozon_ID,Артикул,День,Посетители,Посетители_на_карточке_товара,Показы,Позиция_в_поиске_и_каталоге,В_корзину,...,Трафареты_Клики,Трафареты_Расходы_всего,Трафареты_Количество_заказов,Трафареты_Сумма_заказов,Продвижение_в_поиске_Количество_заказов,Цена_продажи,Продвижение_в_поиске_Сумма_заказов,Ставка_%,Продвижение_в_поиске_Ставка,Контент_Количество_фотографий
191,2609196862022-08-08,Лампа автомобильная,260919686,29539,2022-08-08,1,1,4,4584,0,...,0,0,0,0,0,0,0,0,0,2
192,2609196922022-06-09,Лампа автомобильная,260919692,29539,2022-06-09,1,0,1,4584,0,...,0,0,0,0,0,0,0,0,0,2
193,2609196922022-07-05,Лампа автомобильная,260919692,29539,2022-07-05,1,0,1,4584,0,...,0,0,0,0,0,0,0,0,0,2
194,2609196922022-07-16,Лампа автомобильная,260919692,29539,2022-07-16,1,0,3,4584,0,...,0,0,0,0,0,0,0,0,0,2
195,2609196922022-07-29,Лампа автомобильная,260919692,29539,2022-07-29,1,0,1,4584,0,...,0,0,0,0,0,0,0,0,0,2


In [ ]:
df_svod.to_csv('/content/df_svod.csv',sep=';')

In [ ]:
all_df['Свод'] = df_svod.copy()

In [ ]:
# Сохраняем промежуточный вариант в кэш
df_svod.to_pickle('/content/df_svod.pkl')

In [ ]:
df_svod.dtypes

s_index                                            object
Категория_3го_уровня                               object
Ozon_ID                                            object
Артикул                                            object
День                                       datetime64[ns]
Посетители                                          int64
Посетители_на_карточке_товара                       int64
Показы                                              int64
Позиция_в_поиске_и_каталоге                         int64
В_корзину                                           int64
Заказано                                            int64
Отменено                                            int64
Доставлено                                          int64
Возвращено                                          int64
Оборот_в_месяц_целевой                              int64
Акции                                               int64
Заказано_по_Акции                                   int64
Заказано_на_су

In [ ]:
#from pandas.core.tools.times import time
import time

In [ ]:
# Восстанавливаем из кэша
t = time.time()
df = pd.read_pickle("/content/df_svod.pkl")
tt = time.time()
print(f'замер - {tt - t}')

замер - 0.06255793571472168


In [ ]:
df.dtypes

s_index                                            object
Категория_3го_уровня                               object
Ozon_ID                                            object
Артикул                                            object
День                                       datetime64[ns]
Посетители                                          int64
Посетители_на_карточке_товара                       int64
Показы                                              int64
Позиция_в_поиске_и_каталоге                         int64
В_корзину                                           int64
Заказано                                            int64
Отменено                                            int64
Доставлено                                          int64
Возвращено                                          int64
Оборот_в_месяц_целевой                              int64
Акции                                               int64
Заказано_по_Акции                                   int64
Заказано_на_су

In [ ]:
t = time.time()
df = pd.read_csv('/content/df_svod.csv', delimiter=';')
tt = time.time()
print(f'замер - {tt - t}')

замер - 0.5020630359649658


<ipython-input-170-0d5a6dd9b5ca>:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/df_svod.csv', delimiter=';')


In [ ]:
df.dtypes

Unnamed: 0                                  int64
s_index                                    object
Категория_3го_уровня                       object
Ozon_ID                                     int64
Артикул                                    object
День                                       object
Посетители                                  int64
Посетители_на_карточке_товара               int64
Показы                                      int64
Позиция_в_поиске_и_каталоге                 int64
В_корзину                                   int64
Заказано                                    int64
Отменено                                    int64
Доставлено                                  int64
Возвращено                                  int64
Оборот_в_месяц_целевой                      int64
Акции                                       int64
Заказано_по_Акции                           int64
Заказано_на_сумму_по_акции                  int64
Средний_чек                                 int64


ВЫВОД!!!
Выгружать промежуточные результаты подготовки ДатаФреймов Пандас ЛУЧШЕ функцией:
df.to_pickle('df.pkl')
т.к.:

1. Сохраняется и восстанавливается от 8 до 10 раз быстрее
2. После восстановления не приходится снова устанавливать типы данных в колонках. Всё восстановится 1 к 1

минусы:
1. файлы .pkl возможно использовать только в средах/языках поддерживающих данный кросплатформенный формат
2. результирующий файл Имеет бОльший размер чем CSV


#### MPSTATS (чек-поинт)
продолжаем обогощать Датасет данными
Начинаем с готового Датафрейма собранного по данным ОЗОНа
https://drive.google.com/file/d/1D8hcRg9oNv2FDbEmffCyBBqyF9us47Jj/view?usp=sharing - df_svod.pkl

###### Запускаем, если начинаем с этого места

In [1]:
# Запускаем, если начинаем с этого места
import numpy as np                        # Импорт библиотеки numpy
import pandas as pd                       # для анализа табличных массивов данных
from pandas.core.groupby import NamedAgg
import gdown, os, re, random                  # Стандартные библиотеки

In [2]:
os.chdir('/content')
!gdown --id '1D8hcRg9oNv2FDbEmffCyBBqyF9us47Jj' # df_svod.pkl


/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1D8hcRg9oNv2FDbEmffCyBBqyF9us47Jj
To: /content/df_svod.pkl
100% 68.6M/68.6M [00:00<00:00, 253MB/s]


In [3]:
df_svod = pd.read_pickle("/content/df_svod.pkl")

In [ ]:
!gdown --id '1-_LKbDsiq0dDbks7_JnBJVV7v2iWjSkd'
#!gdown --id '1CzIZl5hxOpNsEiWkcJgjXRsAVJz4RbsX'
!rm -r sample_data
#!rm base_for_analiz.zip
!unzip -o /content/base_for_analiz.zip
# !unzip -o /content/target_tab.zip
# !rm target_tab.zip
!rm base_for_analiz.zip

In [81]:
os.getcwd()

'/content'

In [5]:
def travel_dir(folder_path, subdir=True):
    file_dict = {}
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            file_name = os.path.splitext(file)[0]
            file_dict[file_name] = file_path
        if not subdir:
          return file_dict
    return file_dict


In [6]:
import zipfile
def travel_dir_zip(folder_path):
    # Создаем словарь, в котором будем хранить файлы
    files_dict = {}
    # Обходим папку с архивами
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        # Проверяем, является ли файл zip-архивом
        if zipfile.is_zipfile(file_path):
            # Разархивируем файл
            with zipfile.ZipFile(file_path, 'r') as zip_ref:
                zip_ref.extractall(folder_path)
            # Удаляем файл zip
            os.remove(file_path)
            # Обходим папку с разархивированными файлами
        for extracted_file_name in os.listdir(folder_path):
            extracted_file_path = os.path.join(folder_path, extracted_file_name)
            # Добавляем файл в словарь
            files_dict[extracted_file_name] = extracted_file_path
    return files_dict


In [7]:
import re

def clean_column_names(df):
    cleaned_columns = []

    for column in df.columns:
        cleaned_column = re.sub(r"[^\w\s%]", "", column)  # Оставляем только буквы, цифры и пробелы
        cleaned_column = cleaned_column.replace(" ", "_")  # Заменяем пробелы на "_"
        if cleaned_column[-1] == '_':
            cleaned_column = cleaned_column[:-1]
        cleaned_columns.append(cleaned_column)
    return cleaned_columns
    # df.columns = cleaned_columns
    # return df

def normal_col(df):
  cols = clean_column_names(df)
  print(cols)
  df.set_axis(cols, axis=1, inplace=True)

In [8]:
def float2int(df):
  col_float = list(df.select_dtypes(include=['float64']).columns)
  df.fillna(0, inplace=True)
  for col in col_float:
    df[col]=df[col].round(0).astype('int64')

In [9]:
os.chdir('/content')

In [10]:
mpcat_sel_paths = os.listdir('/content/base_for_analiz/MPSTATS/Категория/MPSTATS по категории (по всем продавцам)')

In [11]:
mpcat_sel_paths

['OZON - Автотовары_Автозапчасти_Автосвет - Товары - 01062022-31082022 06062023.csv.zip',
 'OZON - Автотовары_Автозапчасти_Автосвет - Товары - 01122022-28022023 06062023.csv.zip',
 'OZON - Автотовары_Автозапчасти_Автосвет - Товары - 01092022-30112022 06062023.csv.zip',
 'OZON - Автотовары_Автозапчасти_Автосвет - Товары - 01032023-31052023 06062023.csv.zip']

In [12]:
fdir = '/content/base_for_analiz/MPSTATS/Категория/MPSTATS по категории (по всем продавцам)'
mpcat_sel_paths = travel_dir_zip(fdir)

In [15]:
mpcat_sel_paths

['/content/base_for_analiz/MPSTATS/Категория/MPSTATS по категории (по всем продавцам)/OZON - Автотовары_Автозапчасти_Автосвет - Товары - 01122022-28022023 06062023.csv.zip',
 '/content/base_for_analiz/MPSTATS/Категория/MPSTATS по категории (по всем продавцам)/OZON - Автотовары_Автозапчасти_Автосвет - Товары - 01092022-30112022 06062023.csv.zip',
 '/content/base_for_analiz/MPSTATS/Категория/MPSTATS по категории (по всем продавцам)/OZON - Автотовары_Автозапчасти_Автосвет - Товары - 01062022-31082022 06062023.csv',
 '/content/base_for_analiz/MPSTATS/Категория/MPSTATS по категории (по всем продавцам)/OZON - Автотовары_Автозапчасти_Автосвет - Товары - 01032023-31052023 06062023.csv.zip',
 '/content/base_for_analiz/MPSTATS/Категория/MPSTATS по категории (по всем продавцам)/OZON - Автотовары_Автозапчасти_Автосвет - Товары - 01122022-28022023 06062023.csv',
 '/content/base_for_analiz/MPSTATS/Категория/MPSTATS по категории (по всем продавцам)/OZON - Автотовары_Автозапчасти_Автосвет - Товары - 0

In [14]:
qq = []
if type(mpcat_sel_paths) == dict:
  for i in mpcat_sel_paths:
    qq.append(mpcat_sel_paths[i])
  mpcat_sel_paths = qq.copy()

###### Продлжаем, если начинаем с начала колаба или запущен предыдущий блок

In [16]:
all_cat_sellers = pd.DataFrame()
os.chdir('/content/base_for_analiz/MPSTATS/Категория/MPSTATS по категории (по всем продавцам)')
names = [1,3,5,17]  #["SKU","Категория","Продавец","Был в наличии"]
for i in range(len(mpcat_sel_paths)): # Если тип list Когда продолжаем запущеный с начала колаб

    #clean_csv(mpcat_sel_paths[i])
    try:
      df = pd.read_csv(mpcat_sel_paths[i], na_values='NaN', delimiter=';', decimal=',', usecols=names)
    except:
      continue # отсекаем zip

    float2int(df)
    normal_col(df)

    print('*')
    print('*'*40)
    print(f'-- Текущая таблица: \n {mpcat_sel_paths[i]}', '\n')
    print('---Количество строк: ', df.shape[0], '\n')
    print('--- состав колонок:\n ',df.columns.to_list(), '\n')

    if df.shape[0] > 1:
        all_cat_sellers = pd.concat([all_cat_sellers, df], ignore_index=True)
os.chdir('/content')

['SKU', 'Категория', 'Продавец', 'Был_в_наличии']
*
****************************************
-- Текущая таблица: 
 /content/base_for_analiz/MPSTATS/Категория/MPSTATS по категории (по всем продавцам)/OZON - Автотовары_Автозапчасти_Автосвет - Товары - 01062022-31082022 06062023.csv 

---Количество строк:  80436 

--- состав колонок:
  ['SKU', 'Категория', 'Продавец', 'Был_в_наличии'] 



<ipython-input-7-06ccb71f2ffc>:19: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(cols, axis=1, inplace=True)


['SKU', 'Категория', 'Продавец', 'Был_в_наличии']
*
****************************************
-- Текущая таблица: 
 /content/base_for_analiz/MPSTATS/Категория/MPSTATS по категории (по всем продавцам)/OZON - Автотовары_Автозапчасти_Автосвет - Товары - 01122022-28022023 06062023.csv 

---Количество строк:  138241 

--- состав колонок:
  ['SKU', 'Категория', 'Продавец', 'Был_в_наличии'] 



<ipython-input-7-06ccb71f2ffc>:19: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(cols, axis=1, inplace=True)


['SKU', 'Категория', 'Продавец', 'Был_в_наличии']
*
****************************************
-- Текущая таблица: 
 /content/base_for_analiz/MPSTATS/Категория/MPSTATS по категории (по всем продавцам)/OZON - Автотовары_Автозапчасти_Автосвет - Товары - 01092022-30112022 06062023.csv 

---Количество строк:  142129 

--- состав колонок:
  ['SKU', 'Категория', 'Продавец', 'Был_в_наличии'] 



<ipython-input-7-06ccb71f2ffc>:19: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(cols, axis=1, inplace=True)


['SKU', 'Категория', 'Продавец', 'Был_в_наличии']
*
****************************************
-- Текущая таблица: 
 /content/base_for_analiz/MPSTATS/Категория/MPSTATS по категории (по всем продавцам)/OZON - Автотовары_Автозапчасти_Автосвет - Товары - 01032023-31052023 06062023.csv 

---Количество строк:  177388 

--- состав колонок:
  ['SKU', 'Категория', 'Продавец', 'Был_в_наличии'] 



<ipython-input-7-06ccb71f2ffc>:19: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(cols, axis=1, inplace=True)


In [17]:
all_cat_sellers.count()

SKU              538194
Категория        538194
Продавец         538194
Был_в_наличии    538194
dtype: int64

In [ ]:
def str2datetime(date_str):
  dformat = "Запросы_%Y%m%d"
  date_obj = pd.to_datetime(date_str, format=dformat)
  return date_obj

In [18]:
pd.options.display.width = 3000
pd.options.display.max_columns = 3000
pd.set_option('display.max_columns', 2000)

In [39]:
dfb = all_cat_sellers.copy(deep=True)

In [40]:
print(len(dfb))
dfb['SKU'] = dfb['SKU'].astype(str)
dfb.dtypes

538194


SKU              object
Категория        object
Продавец         object
Был_в_наличии     int64
dtype: object

In [41]:
df_svod.columns

Index(['s_index', 'Категория_3го_уровня', 'Ozon_ID', 'Артикул', 'День', 'Посетители', 'Посетители_на_карточке_товара', 'Показы', 'Позиция_в_поиске_и_каталоге', 'В_корзину', 'Заказано', 'Отменено', 'Доставлено', 'Возвращено', 'Оборот_в_месяц_целевой', 'Акции', 'Заказано_по_Акции', 'Заказано_на_сумму_по_акции', 'Средний_чек', 'Контент_рейтинг', 'Текущая_цена', 'Трафареты_Охват', 'Трафареты_Клики', 'Трафареты_Расходы_всего', 'Трафареты_Количество_заказов', 'Трафареты_Сумма_заказов', 'Продвижение_в_поиске_Количество_заказов', 'Цена_продажи', 'Продвижение_в_поиске_Сумма_заказов', 'Ставка_%', 'Продвижение_в_поиске_Ставка', 'Контент_Количество_фотографий'], dtype='object')

In [42]:
# беру из Свода Ид и категорию, чтобы вычислить количество продавцов в категории
dfs = df_svod[['Ozon_ID','Категория_3го_уровня']].copy(deep=True)
dfb.rename(columns={'SKU':'Ozon_ID'}, inplace=True)
dfb['Ozon_ID'] = dfb['Ozon_ID'].astype(str)
print('Всего уников в своде - ', len(dfs['Ozon_ID'].unique()))
print('Всего уников в Котегориях', len(dfb['Ozon_ID'].unique()))

Всего уников в своде -  1233
Всего уников в Котегориях 326370


In [43]:
# выясняем сколько совпадающих товаров в двух базах
uniq_b = np.intersect1d(dfs['Ozon_ID'].unique(), dfb['Ozon_ID'].unique())
w = uniq_b.tolist()
print('Кол-во строк общих по Ozon_ID - ',len(w))

Кол-во строк общих по Ozon_ID -  894


ЗАДАЧА:
1. Получить для каждого товара (Ozon_ID) количество продавцов в категории этого товара

Наименования категорий различаются, поэтому
придётся с бубном и такой-то матерью выковыривать

1. продавцов по каждой категории. для этого:
1.1. посчитать количество продавцов в каждой категории
1.2. выявить общие ID в таблицах
1.3. объединить с категориями из двух таблиц
1.4. вытащить таблицу соответствия категорий операясь на ID товара
1.5. свести в одну таблицу ['ID', 'КолПродавцов','Был в наличии']

In [24]:
print("Кол-во строк в базе Категорий - ",len(dfb))
print("Кол-во строк в сводной базе   - ",len(dfs))
print('общее кол-во строк -            ', len(dfs)+len(dfb))

Кол-во строк в базе Категорий -  538194
Кол-во строк в сводной базе   -  254920
общее кол-во строк -             793114


In [25]:
print("len(dfb = ",len(dfb))
print("len(dfs) = ",len(dfs))
print('len(dfs)+len(dfb) = ', len(dfs)+len(dfb))

len(dfb =  538194
len(dfs) =  254920
len(dfs)+len(dfb) =  793114


In [44]:
dfb.reset_index()
dfb.loc[dfb.any(axis=1)]

,Ozon_ID,Категория,Продавец,Был_в_наличии
0,268374826,Автотовары/Автозапчасти/Автосвет/Фары,"ООО ""Какие Люди""",3
1,313945368,Автотовары/Автозапчасти/Автосвет/Фары,Ephemeralo,41
2,489815589,Автотовары/Автозапчасти/Автосвет/Фары,sunlinlin,37
3,477676033,Автотовары/Автозапчасти/Автосвет/Фары,Steadfast,80
4,469687960,Автотовары/Автозапчасти/Автосвет/Подсветка,sunlinlin,82
...,...,...,...,...
538189,831903599,Автотовары/Автозапчасти/Автосвет/Автолампы,DawnKnight.Led,90
538190,1004503829,Автотовары/Автозапчасти/Автосвет,LedPlaza,1
538191,914000039,Автотовары/Автозапчасти/Автосвет/Подсветка,CLOZON,61
538192,897231539,Автотовары/Автозапчасти/Автосвет/Комплектующие...,ARUS Маркет,4


Все данные заполнены. пропусков нет

In [45]:
# Избавляемся от неизвестных Продавцов (Возможно и не стоило, но решил так)
# Смотрим, сколько будет удалено
dfb[dfb['Продавец']==0].shape[0]

2593

In [46]:
# Удаляем
dfb = dfb[dfb['Продавец']!=0].copy(deep=True)

In [47]:
dfb.shape[0]

535601

In [48]:
# Получаем количество уникальных конурентов в каждой категории
konkur = dfb.groupby("Категория")["Продавец"].nunique()
konkur = pd.DataFrame(konkur).reset_index()
konkur

,Категория,Продавец
0,Автотовары/Автозапчасти/Автосвет,1379
1,Автотовары/Автозапчасти/Автосвет/Автолампы,3097
2,Автотовары/Автозапчасти/Автосвет/Комплектующие...,391
3,Автотовары/Автозапчасти/Автосвет/Комплектующие...,1217
4,Автотовары/Автозапчасти/Автосвет/Подсветка,2157
5,Автотовары/Автозапчасти/Автосвет/Проблесковые ...,758
6,Автотовары/Автозапчасти/Автосвет/Прожектора,567
7,Автотовары/Автозапчасти/Автосвет/Табло для тра...,22
8,Автотовары/Автозапчасти/Автосвет/Фары,2912


In [49]:
dfb['ID'] = dfb['Ozon_ID'] # чисто для визуальной сверки

In [51]:
# Меняем местами колонки как нам удобно
dfb = dfb.reindex(columns=['Ozon_ID', 'ID', 'Категория','Продавец', 'Был_в_наличии'])

In [52]:
dfb

,Ozon_ID,ID,Категория,Продавец,Был_в_наличии
0,268374826,268374826,Автотовары/Автозапчасти/Автосвет/Фары,"ООО ""Какие Люди""",3
1,313945368,313945368,Автотовары/Автозапчасти/Автосвет/Фары,Ephemeralo,41
2,489815589,489815589,Автотовары/Автозапчасти/Автосвет/Фары,sunlinlin,37
3,477676033,477676033,Автотовары/Автозапчасти/Автосвет/Фары,Steadfast,80
4,469687960,469687960,Автотовары/Автозапчасти/Автосвет/Подсветка,sunlinlin,82
...,...,...,...,...,...
538189,831903599,831903599,Автотовары/Автозапчасти/Автосвет/Автолампы,DawnKnight.Led,90
538190,1004503829,1004503829,Автотовары/Автозапчасти/Автосвет,LedPlaza,1
538191,914000039,914000039,Автотовары/Автозапчасти/Автосвет/Подсветка,CLOZON,61
538192,897231539,897231539,Автотовары/Автозапчасти/Автосвет/Комплектующие...,ARUS Маркет,4


In [53]:
##!!!
#Данный способ, почему-то приводит к увеличению базы, хотя должно быть уменьшение при таком типе объединения. Чудесатые чудеса
df1 = pd.merge(dfb,dfs, on='Ozon_ID', how='inner')
df1.shape

(819820, 6)

In [56]:
# Меняем местами колонки как нам удобно
df1 = df1.reindex(columns=['Ozon_ID', 'ID', 'Категория_3го_уровня', 'Категория','Продавец', 'Был_в_наличии'])

In [57]:
df1

,Ozon_ID,ID,Категория_3го_уровня,Категория,Продавец,Был_в_наличии
0,472486313,472486313,Фара автомобильная,Автотовары/Автозапчасти/Автосвет/Фары,ГАЛОГЕНУ NET,84
1,472486313,472486313,Фара автомобильная,Автотовары/Автозапчасти/Автосвет/Фары,ГАЛОГЕНУ NET,84
2,472486313,472486313,Фара автомобильная,Автотовары/Автозапчасти/Автосвет/Фары,ГАЛОГЕНУ NET,84
3,472486313,472486313,Фара автомобильная,Автотовары/Автозапчасти/Автосвет/Фары,ГАЛОГЕНУ NET,84
4,472486313,472486313,Фара автомобильная,Автотовары/Автозапчасти/Автосвет/Фары,ГАЛОГЕНУ NET,84
...,...,...,...,...,...,...
819815,882353427,882353427,Разное автопринадлежности,Автотовары/Автозапчасти/Автосвет/Подсветка,ГАЛОГЕНУ NET,85
819816,882353427,882353427,Разное автопринадлежности,Автотовары/Автозапчасти/Автосвет/Подсветка,ГАЛОГЕНУ NET,85
819817,882353427,882353427,Разное автопринадлежности,Автотовары/Автозапчасти/Автосвет/Подсветка,ГАЛОГЕНУ NET,85
819818,882353427,882353427,Разное автопринадлежности,Автотовары/Автозапчасти/Автосвет/Подсветка,ГАЛОГЕНУ NET,85


In [58]:
df1.groupby("Категория_3го_уровня")["Продавец"].nunique()

Категория_3го_уровня
Другое                                       1
Комплектующие для ксеноновых фар             1
Комплектующие светодиодной лампы для авто    1
Лампа автомобильная                          1
Разное автопринадлежности                    1
Фара автомобильная                           1
Фонарь задний для авто                       1
Name: Продавец, dtype: int64

Почему-то после объединения, Все продавцы заместились одним. Чудесатые чудеса
Пока не знаю чего с этим делать. не хочу тратить время пока, т.к. это уже не важно. Я уже конкурентов посчитал

In [60]:
df = df1[['Категория_3го_уровня','Категория']].copy(deep=True)

In [62]:
# Запустить 2 раза !!! должно получиться - (7, 2)
# Получаем таблицу соответствия категорий из 2-х систем
df.dropna(subset=['Категория_3го_уровня'], inplace=True) # Почему-то с первого раза не срабатывает ???
df['Категория_3го_уровня'] = df['Категория_3го_уровня'].drop_duplicates()
df.shape

(7, 2)

In [63]:
cat = df[['Категория','Категория_3го_уровня']].copy(deep=True)

#### ИТОГО: Получили две таблицы:
1. **таблицу соответствия Категорий Озон-МПСТАТс**
2. **Количество конкурентов по каждой категории**

**Готов к объединению со сводной таблицей**

In [64]:
print('Таблицу соответствия Категорий Озон-МПСТАТс\n')
display(cat)
print('\n Количество конкуретнов по каждой категории \n')
display(konkur)

Таблицу соответствия Категорий Озон-МПСТАТс



,Категория,Категория_3го_уровня
0,Автотовары/Автозапчасти/Автосвет/Фары,Фара автомобильная
650,Автотовары/Автозапчасти/Автосвет/Комплектующие...,Комплектующие для ксеноновых фар
46939,Автотовары/Автозапчасти/Автосвет/Комплектующие...,Другое
74188,Автотовары/Автозапчасти/Автосвет/Автолампы,Лампа автомобильная
748087,Автотовары/Автозапчасти/Автосвет/Фары,Фонарь задний для авто
810808,Автотовары/Автозапчасти/Автосвет/Автолампы,Комплектующие светодиодной лампы для авто
819732,Автотовары/Автозапчасти/Автосвет/Подсветка,Разное автопринадлежности



 Количество конкуретнов по каждой категории 



,Категория,Продавец
0,Автотовары/Автозапчасти/Автосвет,1379
1,Автотовары/Автозапчасти/Автосвет/Автолампы,3097
2,Автотовары/Автозапчасти/Автосвет/Комплектующие...,391
3,Автотовары/Автозапчасти/Автосвет/Комплектующие...,1217
4,Автотовары/Автозапчасти/Автосвет/Подсветка,2157
5,Автотовары/Автозапчасти/Автосвет/Проблесковые ...,758
6,Автотовары/Автозапчасти/Автосвет/Прожектора,567
7,Автотовары/Автозапчасти/Автосвет/Табло для тра...,22
8,Автотовары/Автозапчасти/Автосвет/Фары,2912


In [65]:
cat.set_index(['Категория'])

,Категория_3го_уровня
Категория,
Автотовары/Автозапчасти/Автосвет/Фары,Фара автомобильная
Автотовары/Автозапчасти/Автосвет/Комплектующие для ксенона,Комплектующие для ксеноновых фар
Автотовары/Автозапчасти/Автосвет/Комплектующие для ксенона,Другое
Автотовары/Автозапчасти/Автосвет/Автолампы,Лампа автомобильная
Автотовары/Автозапчасти/Автосвет/Фары,Фонарь задний для авто
Автотовары/Автозапчасти/Автосвет/Автолампы,Комплектующие светодиодной лампы для авто
Автотовары/Автозапчасти/Автосвет/Подсветка,Разное автопринадлежности


In [66]:
konkur

,Категория,Продавец
0,Автотовары/Автозапчасти/Автосвет,1379
1,Автотовары/Автозапчасти/Автосвет/Автолампы,3097
2,Автотовары/Автозапчасти/Автосвет/Комплектующие...,391
3,Автотовары/Автозапчасти/Автосвет/Комплектующие...,1217
4,Автотовары/Автозапчасти/Автосвет/Подсветка,2157
5,Автотовары/Автозапчасти/Автосвет/Проблесковые ...,758
6,Автотовары/Автозапчасти/Автосвет/Прожектора,567
7,Автотовары/Автозапчасти/Автосвет/Табло для тра...,22
8,Автотовары/Автозапчасти/Автосвет/Фары,2912


In [67]:
df = pd.merge(cat,konkur, on='Категория', how='left') # , left_index='Категория', right_index='Категория'

In [68]:
df

,Категория,Категория_3го_уровня,Продавец
0,Автотовары/Автозапчасти/Автосвет/Фары,Фара автомобильная,2912
1,Автотовары/Автозапчасти/Автосвет/Комплектующие...,Комплектующие для ксеноновых фар,391
2,Автотовары/Автозапчасти/Автосвет/Комплектующие...,Другое,391
3,Автотовары/Автозапчасти/Автосвет/Автолампы,Лампа автомобильная,3097
4,Автотовары/Автозапчасти/Автосвет/Фары,Фонарь задний для авто,2912
5,Автотовары/Автозапчасти/Автосвет/Автолампы,Комплектующие светодиодной лампы для авто,3097
6,Автотовары/Автозапчасти/Автосвет/Подсветка,Разное автопринадлежности,2157


In [ ]:
df.columns

Index(['Категория', 'Категория_3го_уровня', 'Продавец'], dtype='object')

In [69]:
df.drop('Категория', axis=1, inplace=True)
df.columns

Index(['Категория_3го_уровня', 'Продавец'], dtype='object')

In [70]:
# Меняем местами колонки как нам удобно
df.rename(columns={'Продавец':'Конкуренты_в_категории'}, inplace=True)

In [73]:
df.sort_values(by='Категория_3го_уровня', inplace=True)
df.reset_index()
df

,Категория_3го_уровня,Конкуренты_в_категории
2,Другое,391
1,Комплектующие для ксеноновых фар,391
5,Комплектующие светодиодной лампы для авто,3097
3,Лампа автомобильная,3097
6,Разное автопринадлежности,2157
0,Фара автомобильная,2912
4,Фонарь задний для авто,2912


In [76]:
df_svod.head()

,s_index,Категория_3го_уровня,Ozon_ID,Артикул,День,Посетители,Посетители_на_карточке_товара,Показы,Позиция_в_поиске_и_каталоге,В_корзину,Заказано,Отменено,Доставлено,Возвращено,Оборот_в_месяц_целевой,Акции,Заказано_по_Акции,Заказано_на_сумму_по_акции,Средний_чек,Контент_рейтинг,Текущая_цена,Трафареты_Охват,Трафареты_Клики,Трафареты_Расходы_всего,Трафареты_Количество_заказов,Трафареты_Сумма_заказов,Продвижение_в_поиске_Количество_заказов,Цена_продажи,Продвижение_в_поиске_Сумма_заказов,Ставка_%,Продвижение_в_поиске_Ставка,Контент_Количество_фотографий
0,2609022712022-06-02,Лампа автомобильная,260902271,25541,2022-06-02,1,0,1,4584,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2609022712022-06-03,Лампа автомобильная,260902271,25541,2022-06-03,1,0,1,4584,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2609022712022-06-07,Лампа автомобильная,260902271,25541,2022-06-07,1,1,1,4584,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2609022712022-06-08,Лампа автомобильная,260902271,25541,2022-06-08,0,0,0,4584,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2609022712022-06-12,Лампа автомобильная,260902271,25541,2022-06-12,1,0,1,4584,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [77]:
df = pd.merge(df_svod,df, on='Категория_3го_уровня', how='left')

In [78]:
df

,s_index,Категория_3го_уровня,Ozon_ID,Артикул,День,Посетители,Посетители_на_карточке_товара,Показы,Позиция_в_поиске_и_каталоге,В_корзину,Заказано,Отменено,Доставлено,Возвращено,Оборот_в_месяц_целевой,Акции,Заказано_по_Акции,Заказано_на_сумму_по_акции,Средний_чек,Контент_рейтинг,Текущая_цена,Трафареты_Охват,Трафареты_Клики,Трафареты_Расходы_всего,Трафареты_Количество_заказов,Трафареты_Сумма_заказов,Продвижение_в_поиске_Количество_заказов,Цена_продажи,Продвижение_в_поиске_Сумма_заказов,Ставка_%,Продвижение_в_поиске_Ставка,Контент_Количество_фотографий,Конкуренты_в_категории
0,2609022712022-06-02,Лампа автомобильная,260902271,25541,2022-06-02,1,0,1,4584,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3097.0
1,2609022712022-06-03,Лампа автомобильная,260902271,25541,2022-06-03,1,0,1,4584,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3097.0
2,2609022712022-06-07,Лампа автомобильная,260902271,25541,2022-06-07,1,1,1,4584,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3097.0
3,2609022712022-06-08,Лампа автомобильная,260902271,25541,2022-06-08,0,0,0,4584,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3097.0
4,2609022712022-06-12,Лампа автомобильная,260902271,25541,2022-06-12,1,0,1,4584,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3097.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254915,9660743052023-05-28,Лампа автомобильная,966074305,34599,2023-05-28,19,0,19,132,0,0,0,0,0,0,0,0,0,0,40,1640,0,0,0,0,0,0,0,0,0,0,3,3097.0
254916,9660743052023-05-29,Лампа автомобильная,966074305,34599,2023-05-29,26,1,28,155,0,0,0,0,0,0,0,0,0,0,40,1640,0,0,0,0,0,0,0,0,0,0,3,3097.0
254917,9660743052023-05-30,Лампа автомобильная,966074305,34599,2023-05-30,36,3,43,59,0,0,0,0,0,0,0,0,0,0,40,1640,0,0,0,0,0,0,0,0,0,0,3,3097.0
254918,9660743052023-05-31,Лампа автомобильная,966074305,34599,2023-05-31,21,3,30,144,1,0,0,0,0,0,0,0,0,0,40,1640,0,0,0,0,0,0,0,0,0,0,3,3097.0


In [80]:
print(f'Размерность свода:                   {df_svod.shape}')
print(f'Размерность свода после объединения: {df.shape}')

Размерность свода:                   (254920, 32)
Размерность свода после объединения: (254920, 33)


ИТОГ:
Количество конкурентов в Свод добавил. С фичой "Был в наличии" пока не ясно. Оставляю до прояснения ситуации. Не понятно ее целесообразность и применимость по логике

In [82]:
# Сохраняем промежуточно-окончательный вариант в кэш
df.to_pickle('/content/df_svod.pkl')